# CTCF feature extraction

requirements
* try: 40 bp window on left, 40-80 bp window on after motif on right (don't need to go 100s bp out)
* within an MSP [centered_start <= 0, centered_end >= 35]
* remove observations with N flanking or in the motif
* no FDR filtering 

potential features
* [x] read length [query_length]
* [x] 3-mer k-mer count within canonical motif (exclude k-mers without AT) [24 possible 3-mers]
* [x] m6a proportion/total possible m6a instances per k-mer
* [x] m6a count for: (motif, 40 bp flank left, 40 bp flank right)
* [x] AT count for: (motif, flank left, flank right)
* [x] proportion of methylated ATs per element (motif, flank left, flank right) (0-1)
* [x] proportion of bases that are AT per element (0-1)
* [x] total methylation across the read (% of ATs that were methylated across read) [use subseq_sequence for now]
* [x] MSP size
* [ ] do run length encoding of motif in AT space

#### table interpretation

in matrix:
* each row reflects 1 prediction we want
* i.e. every row will be all the info we want for a group_by done on motif_name (chrom, centering_position, strand) & query_name
* per query_name, per motif_name: predict if a query_name at a site (motif) is bound by a CTCF

| col num | col | meaning |
| :---: | :-- | :-- |
| 0 | chrom | chromosome |
| 1 | centering_position | motif start when aligned to reference |
| 2 | strand | strand |
| 3 | subset_sequence | sequence flanking motif start x bps on each side |
| 4 | reference_start | starting position of HiFi read when aligned to reference |
| 5 | reference_end | ending position of HiFi read |
| 6 | query_name | fiber (or sequencing read) name |
| 7 | centered_query_start | HiFi read start relative to motif start |
| 8 | centered_query_end | HiFi read end relative to motif start |
| 9 | query_length | HiFi read length (~20,000 bps) |
| 10 | centered_position_type | event type |
| 11 | centered_start | distance of event start to centering position |
| 12 | centered_end | distance of event end to centering position |

data interpratation
* unique motifs: same chrom, centering_position, strand
* fiber (sequencing read): all elements with the same query_name
* to get m6a events over a single fiber on a single motif: group by the first 3 columns and the query_name
* same chrom, centering_position, strand, & query_name: should have the same subset_sequence
* subset_sequence is not the same for the same motif. m6a mods & sequence come from the sequencing read named in query_name & genetic_variation is expected

#### output table

* rows: each prediction group (motif-query_name)
* columns: features
* 2 k-mer cols: a. count of k-mer within the motif of the row, b. m6a count within k-mer

| group (chrom, centering_position, strand, query_name) | features | m6a count left |
| :---: | :-- | :-- |
| prediction group | ... | ... |

In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
from collections import defaultdict
from collections import Counter
import itertools

In [3]:
import functools
import itertools
from collections import defaultdict
from collections import Counter

### workflow
1. read in data
2. filter for only m6a and msp rows
3. make rows canonical
4. add motif names
5. filter for MSPs with a motif (centered_start < 0, centered_end > 35)
6. remove MSPs without a motif
7. add msp length
8. filter for m6as flanking 40bp (centered_start >= -40, centered_end < 75)
9. group rows by first 6 columns (chrom, centering_position, strand, reference_start, reference_end, query_name)

In [4]:
# M, L, XL
print("Summed motifs: {:,}".format(20053779 + 12739713 + 10967458))
print("First merged: {:,}".format(30490055))
print("merged v2: {:,}".format(43760948))

Summed motifs: 43,760,950
First merged: 30,490,055
merged v2: 43,760,948


In [21]:
# set folders
project_dir = "/mmfs1/gscratch/stergachislab/mwperez/ctcf-footprinting"
data_dir = "{}/candidate_footprints".format(project_dir)
output_dir = "{}/feature_data".format(project_dir)

In [6]:
def clean_sequences(df):
    '''Remove rows with N characters.'''
    print("Removing "+ "{:,} rows.".format(df["subset_sequence"].str.contains("N").sum()))
    return df[~df["subset_sequence"].str.contains("N")]

##### positive data (merged)

In [53]:
# CTCF merged
motif_type = "merged"
data_file = "{}/CTCF_m6a_fiberseq_100bp.txt".format(data_dir)

In [54]:
%%time
# read in data
n_rows = None
df = pd.read_csv(data_file, sep="\t", nrows=n_rows)
print(f"{df.shape[0]:,d}")

# filter for only m6a & msp rows
df = df[df["centered_position_type"].isin(["m6a", "msp"])]
print(f"{df.shape[0]:,d}")

30,490,054
28,275,440
CPU times: user 48.4 s, sys: 6.67 s, total: 55 s
Wall time: 55.2 s


In [55]:
# add col to df of unique motifs names
df.insert(loc=0, column="motif_name", 
          value=df["chrom"]+"_"+df["centering_position"].astype(str)+"_"+df["strand"].astype(str))

In [56]:
print("Columns: {}".format(df.shape[1]))
print("Total rows: " + "{:,}".format(df.shape[0]))
print("Total unique motifs: {:,}".format(df.motif_name.nunique()))
print("Total unique query_names/strands: " + "{:,}".format(df.query_name.nunique()))

print("MSP's: " + "{:,}".format(df[df["centered_position_type"] == "msp"].shape[0]))
print("m6a's: " + "{:,}".format(df[df["centered_position_type"] == "m6a"].shape[0]))

Columns: 14
Total rows: 28,275,440
Total unique motifs: 15762
Total unique query_names/strands: 623,486
MSP's: 952,272
m6a's: 27,323,168


In [57]:
# remove rows with Ns in sequence
df = clean_sequences(df)
print(f"{df.shape[0]:,d}")

Removing 271,779 rows.
28,003,661


##### positive data (L)

In [3]:
# CTCF L (best footprint according to Andrew)
motif_type = "L"
data_file = "{}/CTCF_m6a_fiberseq_{}_100bp.txt".format(data_dir, motif_type)

In [6]:
%%time
# read in data
n_rows = None
df = pd.read_csv(data_file, sep="\t", nrows=n_rows)
print(f"{df.shape[0]:,d}")

# filter for only m6a & msp rows
df = df[df["centered_position_type"].isin(["m6a", "msp"])]
print(f"{df.shape[0]:,d}")

12,739,712
11,860,730
CPU times: user 19.7 s, sys: 2.61 s, total: 22.4 s
Wall time: 22.4 s


In [8]:
print("Columns: {}".format(df.shape[1]))
print("Total rows: " + "{:,}".format(df.shape[0]))
print("Total unique motifs: {}".format(df.motif_name.nunique()))
print("Total unique query_names/strands: " + "{:,}".format(df.query_name.nunique()))

print("MSP's: " + "{:,}".format(df[df["centered_position_type"] == "msp"].shape[0]))
print("m6a's: " + "{:,}".format(df[df["centered_position_type"] == "m6a"].shape[0]))

Columns: 14
Total rows: 11,860,730
Total unique motifs: 6504
Total unique query_names/strands: 328,835
MSP's: 396,372
m6a's: 11,464,358


##### negative data (merged)

In [69]:
# negative CTCF merged
motif_type = "merged"
data_file = "{}/CTCF_negative_m6a_fiberseq_{}_100bp_small.txt".format(data_dir, motif_type)

In [70]:
%%time
# read in data
n_rows = None
df = pd.read_csv(data_file, sep="\t", nrows=n_rows)
print(f"{df.shape[0]:,d}")

# filter for only m6a & msp rows
df = df[df["centered_position_type"].isin(["m6a", "msp"])]
print(f"{df.shape[0]:,d}")

25,975,749
20,776,305
CPU times: user 40.7 s, sys: 5.02 s, total: 45.7 s
Wall time: 45.9 s


In [71]:
# add col to df of unique motifs names
df.insert(loc=0, column="motif_name", 
          value=df["chrom"]+"_"+df["centering_position"].astype(str)+"_"+df["strand"].astype(str))

In [72]:
print("Columns: {}".format(df.shape[1]))
print("Total rows: " + "{:,}".format(df.shape[0]))
print("Total unique motifs: {:,}".format(df.motif_name.nunique()))
print("Total unique query_names/strands: " + "{:,}".format(df.query_name.nunique()))

print("MSP's: " + "{:,}".format(df[df["centered_position_type"] == "msp"].shape[0]))
print("m6a's: " + "{:,}".format(df[df["centered_position_type"] == "m6a"].shape[0]))

Columns: 14
Total rows: 20,776,305
Total unique motifs: 1,027,696
Total unique query_names/strands: 93,458
MSP's: 1,762,860
m6a's: 19,013,445


In [73]:
# remove rows with Ns in sequence
df = clean_sequences(df)
print(f"{df.shape[0]:,d}")

Removing 138,550 rows.
20,637,755


##### negative data (L)

In [8]:
# negative CTCF L
motif_type = "L"
data_file = "{}/CTCF_negative_m6a_fiberseq_{}_100bp_small.txt".format(data_dir, motif_type)

In [36]:
%%time
# read in data
n_rows = None
df = pd.read_csv(data_file, sep="\t", nrows=n_rows)
print("ft center data - rows: {:,}, columns: {:,}".format(df.shape[0], df.shape[1]))

# filter for only m6a & msp rows
df = df[df["centered_position_type"].isin(["m6a", "msp"])]
print(f"{df.shape[0]:,d}")

7,878,026
6,405,114
CPU times: user 12.6 s, sys: 1.78 s, total: 14.3 s
Wall time: 14.4 s


In [37]:
# save original table
df_og = df.copy(deep=True)

In [65]:
df = df_og.copy(deep=True)

In [66]:
df.head(1)

,chrom,centering_position,strand,subset_sequence,reference_start,reference_end,query_name,centered_query_start,centered_query_end,query_length,centered_position_type,centered_start,centered_end
0,chr1,11256,-,TGCCAGCAGGCGGCGTGCCACCACTATACAGTAAGCAAGAGGGCCC...,10000,26121,m54329U_210810_004956/34799917/ccs,-6067,14864,20931,m6a,-81,-80


In [94]:
def make_IDs(df, col_name):
    '''Makes a column of columns joined by a string.'''
    if col_name == "motif_name":
        # add column to df of unique motif names
        df.insert(loc=0, column=col_name, value=df["chrom"]+"_"+df["centering_position"].astype(str)+"_"+df["strand"].astype(str))
    else:
        # add column of combined motif/query names
        df.insert(loc=0, column=col_name, value=df["motif_name"]+"/"+df["query_name"])


In [68]:
%%time
make_IDs(df, "motif_name")

CPU times: user 2.61 s, sys: 794 ms, total: 3.41 s
Wall time: 3.38 s


In [69]:
df.head()

,motif_name,chrom,centering_position,strand,subset_sequence,reference_start,reference_end,query_name,centered_query_start,centered_query_end,query_length,centered_position_type,centered_start,centered_end
0,chr1_11256_-,chr1,11256,-,TGCCAGCAGGCGGCGTGCCACCACTATACAGTAAGCAAGAGGGCCC...,10000,26121,m54329U_210810_004956/34799917/ccs,-6067,14864,20931,m6a,-81,-80
1,chr1_11256_-,chr1,11256,-,TGCCAGCAGGCGGCGTGCCACCACTATACAGTAAGCAAGAGGGCCC...,10000,26121,m54329U_210810_004956/34799917/ccs,-6067,14864,20931,m6a,-78,-77
2,chr1_11256_-,chr1,11256,-,TGCCAGCAGGCGGCGTGCCACCACTATACAGTAAGCAAGAGGGCCC...,10000,26121,m54329U_210810_004956/34799917/ccs,-6067,14864,20931,m6a,-75,-74
3,chr1_11256_-,chr1,11256,-,TGCCAGCAGGCGGCGTGCCACCACTATACAGTAAGCAAGAGGGCCC...,10000,26121,m54329U_210810_004956/34799917/ccs,-6067,14864,20931,m6a,-74,-73
4,chr1_11256_-,chr1,11256,-,TGCCAGCAGGCGGCGTGCCACCACTATACAGTAAGCAAGAGGGCCC...,10000,26121,m54329U_210810_004956/34799917/ccs,-6067,14864,20931,m6a,-73,-72


negative dataset vs positive dataset
* a lot more unique motifs than the positive dataset (317,997 vs 6,504)
* but less unique query_names/strands (86,064 vs 328,835)

In [72]:
def print_df_info(df):
    '''Print df content and shape info.'''
    print("Columns: {:,}".format(df.shape[1]))
    print("Total rows: " + "{:,}".format(df.shape[0]))
    print("Total unique motifs: {:,}".format(df.motif_name.nunique()))
    print("Total unique query_names: " + "{:,}".format(df.query_name.nunique()))

    print("MSP's: " + "{:,}".format(df[df["centered_position_type"] == "msp"].shape[0]))
    print("m6a's: " + "{:,}".format(df[df["centered_position_type"] == "m6a"].shape[0]))

In [74]:
%%time
print_df_info(df)

Columns: 14
Total rows: 6,405,114
Total unique motifs: 317,997
Total unique query_names: 86,064
MSP's: 543,117
m6a's: 5,861,997
CPU times: user 1.82 s, sys: 148 ms, total: 1.97 s
Wall time: 1.97 s


In [76]:
# remove rows with Ns in sequence
df = clean_sequences(df)
print("Cleaned rows: {:,}".format(df.shape[0]))

Removing 43,252 rows.
6,361,862


### convert rows to canonical strand (ignore for now)

how does it affect the rest of the info? (subset sequence, centered_start, centered_end)

In [ ]:
print(f"{df.shape[0]:,d}")

In [ ]:
df.head(1)

In [ ]:
def clean_sequences(df):
    '''Remove rows with N characters.'''
    print("Removing {} rows.".format(df["subset_sequence"].str.contains("N").sum()))
    return df[~df["subset_sequence"].str.contains("N")]

In [ ]:
def rev_comp(seq):
    ''' Return the reverse complement of the input DNA sequence. '''
    rc_dict = {
        "A": "T",
        "C": "G",
        "G": "C",
        "T": "A"
    }
    rc_seq = "".join(rc_dict[base] for base in seq[::-1])
    return rc_seq

def row_to_canonical(row):
    ''' Convert to canonical strand. '''
    if row["strand"] == "+":
        row = row
    else:
        # if strand is "-"
        row["strand"] = "+"
        row["subset_sequence"] = rev_comp(row["subset_sequence"])
    return row

In [ ]:
# remove rows with Ns in sequence
df = clean_sequences(df)

In [ ]:
print(f"{df.shape[0]:,d}")

In [ ]:
%%time
# get canonical cleaned df
print("Converting to canonical strand.")
df = df.apply(row_to_canonical, axis=1)
print(f"{df.shape[0]:,d}")

### filter for regions within an msp & add msp size

might not be interested in msp's < 85 bp in length

#### issue with removing MSPs that do not contain a motif but are in a motif-query group that does

<font color="red">__Cannot filter out 9 msp regions!!! No idea why!__</font>
<br> indices: [16630, 20018, 48592, 52137, 69538, 75758, 93225, 93459, 95475]

| index | motif_name | query_name | centered_start | centered_end |
| --: | --: | --- | --- | --- |
| 16630 | chr1_3453274_+ | m64076_221119_202646/138085724/ccs | -188 | -99 |
| 20018 | chr1_3673515_- | m54329U_210323_190418/25755973/ccs | -91 | -80 |
| 48592 | chr1_9067419_+ | m64076_221119_202646/138085724/ccs | -212 | -74 |
| 52137 | chr1_9267338_- | m54329U_210814_130637/170459707/ccs | -94 | -92 |
| 69538 | chr1_11650573_+ | m54329U_210323_190418/61605281/ccs | -197 | -85 |
| 75758 | chr1_11907872_- | m64076_210328_012155/88474236/ccs | -100 | -99 |
| 93225 | chr1_14850809_- | m54329U_210326_192251/66061064/ccs | -174 | -92 |
| 93459 | chr1_14850809_- | m64076_221119_202646/111477920/ccs | -205 | -89 |
| 95475 | chr1_15100575_- | m54329U_210323_190418/46138317/ccs | -231 | -90 |

In [138]:
df.loc[np.setdiff1d(df.index.tolist(), idx_keep).tolist()][["motif_name", "query_name", "centered_start", "centered_end"]]

,motif_name,query_name,centered_start,centered_end
16630,chr1_3453274_+,m64076_221119_202646/138085724/ccs,-188,-99
20018,chr1_3673515_-,m54329U_210323_190418/25755973/ccs,-91,-80
48592,chr1_9067419_+,m54329U_210813_020940/46333996/ccs,-212,-74
52137,chr1_9267338_-,m54329U_210814_130637/170459707/ccs,-94,-92
69538,chr1_11650573_+,m54329U_210323_190418/61605281/ccs,-197,-85
75758,chr1_11907872_-,m64076_210328_012155/88474236/ccs,-100,-99
93225,chr1_14850809_-,m54329U_210326_192251/66061064/ccs,-174,-92
93459,chr1_14850809_-,m64076_221119_202646/111477920/ccs,-205,-89
95475,chr1_15100575_-,m54329U_210323_190418/46138317/ccs,-231,-90


Fibers (query_name) have different sequences at different motifs

In [10]:
print(f"{df.shape[0]:,d}")

90,732


In [68]:
# MSP rows
df_msp = df[(df["centered_position_type"] == "msp")]
print("MSP's: " + "{:,}".format(df_msp.shape[0]))

MSP's: 3,302


In [71]:
# mask for motifs within an msp
motif_len = 35
msp_mask = (df["centered_position_type"] == "msp") & (df["centered_start"] <= 0) & (df["centered_end"] >= motif_len)
print("MSP's with a motif: " + "{:,}".format(msp_mask.sum()))
msp_groups = [(row["motif_name"], row["query_name"]) for idx, row in df[msp_mask].iterrows()]

# filter for rows with motifs within an MSP (gets both msp's & m6a's)
df = df[df[["motif_name", "query_name"]].apply(tuple, 1).isin(msp_groups)]
print("Total observations from motifs within an msp: " + "{:,}".format(df.shape[0]))

MSP's with a motif: 1,986
Total observations from motifs within an msp: 73,671


In [74]:
sum((df["centered_position_type"] == "msp") & (df["centered_start"] <= 0) & (df["centered_end"] >= motif_len))

1986

In [ ]:
# remove MSP's not containing a motif
print("Removing MSP's not containing a motif: " + "{:,}".format(
    (~np.logical_or(msp_mask, (df["centered_position_type"] == "m6a"))).sum()))
#df = df[np.logical_or(msp_mask, (df["centered_position_type"] == "m6a"))]

In [82]:
sum(msp_mask)

1986

In [87]:
len(df[df["centered_position_type"] == "m6a"].index.tolist())

71676

In [88]:
df.shape[0]

73671

In [84]:
df[msp_mask].index.tolist()

/tmp/ipykernel_77479/2968305451.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df[msp_mask].index.tolist()


[33,
 55,
 152,
 188,
 217,
 248,
 262,
 305,
 349,
 410,
 453,
 504,
 547,
 594,
 630,
 664,
 707,
 734,
 776,
 797,
 825,
 857,
 894,
 975,
 999,
 1042,
 1098,
 1128,
 1160,
 1202,
 1225,
 1337,
 1396,
 1437,
 1526,
 1589,
 1624,
 1731,
 1771,
 1794,
 1867,
 1917,
 1961,
 2009,
 2064,
 2213,
 2294,
 2392,
 2440,
 2491,
 2555,
 2590,
 2647,
 2715,
 2751,
 2782,
 2824,
 2913,
 2952,
 2998,
 3039,
 3084,
 3136,
 3180,
 3233,
 3279,
 3313,
 3361,
 3396,
 3424,
 3466,
 3546,
 3600,
 3696,
 3734,
 3745,
 3798,
 3853,
 3927,
 3969,
 4048,
 4089,
 4116,
 4158,
 4206,
 4248,
 4291,
 4344,
 4361,
 4387,
 4430,
 4457,
 4499,
 4547,
 4602,
 4672,
 4701,
 4781,
 4842,
 4880,
 4926,
 4969,
 5019,
 5076,
 5109,
 5168,
 5278,
 5380,
 5448,
 5485,
 5519,
 5599,
 5637,
 5690,
 5728,
 5786,
 5838,
 5932,
 5961,
 6029,
 6067,
 6128,
 6175,
 6223,
 6271,
 6313,
 6396,
 6505,
 6553,
 6609,
 6667,
 6707,
 6760,
 6803,
 6828,
 6888,
 6929,
 6978,
 7047,
 7135,
 7224,
 7295,
 7350,
 7418,
 7453,
 7501,
 7565

In [96]:
df[df["centered_position_type"] == "m6a"].index.tolist()[-5:]

[99987, 99988, 99989, 99990, 99991]

In [111]:
print(df.shape)
msp_mask = (df["centered_position_type"] == "msp") & (df["centered_start"] <= 0) & (df["centered_end"] >= motif_len)
print(len(msp_mask))
print(sum(msp_mask))

(73671, 14)
73671
1986


In [116]:
idx_keep = list(df[df["centered_position_type"] == "m6a"].index) + list(df[msp_mask].index)
print(len(idx_keep))

73662


In [127]:
np.setdiff1d(df.index.tolist(), idx_keep).tolist()

[16630, 20018, 48592, 52137, 69538, 75758, 93225, 93459, 95475]

In [135]:
df.loc[np.setdiff1d(df.index.tolist(), idx_keep).tolist()][["centered_end"]] >= 35

,centered_end
16630,False
20018,False
48592,False
52137,False
69538,False
75758,False
93225,False
93459,False
95475,False


In [121]:
df.loc[idx_keep]

TypeError: bad operand type for unary ~: 'list'

In [97]:
df.iloc[idx_keep]

IndexError: positional indexers are out-of-bounds

In [76]:
df[(df["centered_position_type"] == "msp") & (df["centered_end"] < 35)]

,motif_name,chrom,centering_position,strand,subset_sequence,reference_start,reference_end,query_name,centered_query_start,centered_query_end,query_length,centered_position_type,centered_start,centered_end
16630,chr1_3453274_+,chr1,3453274,+,TGTCCCCTCACTGGGCCTCAGTGACCCCACCGCACCTCAGTCCCCT...,3452462,3474434,m64076_221119_202646/138085724/ccs,-812,21148,21960,msp,-188,-99
20018,chr1_3673515_-,chr1,3673515,-,ACGGGTGCGTCGGGTGGAGACGGCGCGAGATCGACACACTGGAGTC...,3659531,3677673,m54329U_210323_190418/25755973/ccs,-14001,4160,18161,msp,-91,-80
48592,chr1_9067419_+,chr1,9067419,+,AGAGGAGGCTTCCTTCCCTGGAGGGCTGAGGCAGCAGGCAGCTGGG...,9063997,9088265,m54329U_210813_020940/46333996/ccs,-3418,20865,24283,msp,-212,-74
52137,chr1_9267338_-,chr1,9267338,-,CACAACGTTGACCTCTCCTCAGCCAAGGACTTCTTGTCTACCCTCT...,9254219,9276385,m54329U_210814_130637/170459707/ccs,-13099,9070,22169,msp,-94,-92
69538,chr1_11650573_+,chr1,11650573,+,CACGGGTTACGCAGAAGGTTGCGGCGCCGCTTGCTCAGGAAGTAGA...,11649474,11659539,m54329U_210323_190418/61605281/ccs,-1100,8984,10084,msp,-197,-85
75758,chr1_11907872_-,chr1,11907872,-,TGCTTGTAATAAATCACGCGTTACGTAACACCCATACGTTTATTTC...,11896836,11908609,m64076_210328_012155/88474236/ccs,-11053,738,11791,msp,-100,-99
93225,chr1_14850809_-,chr1,14850809,-,AAGGAAGACTCTCGACCCGATCCTTTGGTTATATTTTGAAGGTCTC...,14838159,14855962,m54329U_210326_192251/66061064/ccs,-12691,5173,17864,msp,-174,-92
93459,chr1_14850809_-,chr1,14850809,-,CAAAGGAAGACTCTCGACCCGATCCTTTGGTTATATTTTGAAGGTC...,14843213,14866523,m64076_221119_202646/111477920/ccs,-7674,15854,23528,msp,-205,-89
95475,chr1_15100575_-,chr1,15100575,-,CTAAAGGAACACGGGGTGTATCTTTACTGGGGACATGGTGCATATG...,15092884,15100964,m54329U_210323_190418/46138317/ccs,-7717,390,8107,msp,-231,-90


In [79]:
df[df["centered_position_type"] == "msp"]

,motif_name,chrom,centering_position,strand,subset_sequence,reference_start,reference_end,query_name,centered_query_start,centered_query_end,query_length,centered_position_type,centered_start,centered_end
33,chr1_1033080_+,chr1,1033080,+,GACCTACGGGGGCGGGTGTGGGGACGCCGGACTACGCGTCAGGAGT...,1008277,1033947,m54329U_210813_020940/24183559/ccs,-24816,861,25677,msp,-78,147
55,chr1_1033080_+,chr1,1033080,+,GACCTACGGGGGCGGGTGTGGGGACGCCGGACTACGCGTCAGGAGT...,1010739,1033452,m54329U_210813_020940/173737659/ccs,-22317,372,22689,msp,-42,58
152,chr1_1033080_+,chr1,1033080,+,CCTACGGGGGCGGGTGTGGGGACGCCGGAACTTACGCGTCAGGAGT...,1016995,1038612,m54329U_210323_190418/65799401/ccs,-18781,5567,24348,msp,-80,303
188,chr1_1033080_+,chr1,1033080,+,GACCTACGGGGGCGGGTGTGGGGACGCCGGACTACGCGTCAGGAGT...,1017128,1037380,m54329U_210814_130637/153158045/ccs,-15970,4300,20270,msp,-227,219
217,chr1_1033080_+,chr1,1033080,+,GACCTACGGGGGCGGGTGTGGGGACGCCGGACCTACGCGTCAGAGT...,1017682,1033494,m54329U_210323_190418/28969124/ccs,-15438,416,15854,msp,-71,220
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99788,chr1_15792178_+,chr1,15792178,+,AGGGGGACGGTCCCTGCCTAGGGTCCCCTGCGCAGCCTGGGAAGGC...,15777436,15796646,m54329U_210814_130637/115999192/ccs,-14771,4457,19228,msp,-67,99
99843,chr1_15792178_+,chr1,15792178,+,AGGGGGACGGTCCCTGCCTAGGGTCCCCTGCGCAGCCTGGGAAGGC...,15777763,15794222,m54329U_210323_190418/179176258/ccs,-14469,2035,16504,msp,-81,126
99896,chr1_15792178_+,chr1,15792178,+,AGGGGGACGGTCCCTGCCTAGGGTCCCCTGCGCAGCCTGGGAAGGC...,15778376,15806996,m64076_221119_202646/81527990/ccs,-13841,14819,28660,msp,-81,104
99943,chr1_15792178_+,chr1,15792178,+,AGGGGGACGGTCCCTGCCTAGGGTCCCCTGCGCAGCCTGGGAAGGC...,15779324,15792995,m54329U_210326_192251/126943836/ccs,-12896,806,13702,msp,-54,99


In [77]:
df[(df["centered_position_type"] == "msp") & (df["centered_end"] >= 35)]

,motif_name,chrom,centering_position,strand,subset_sequence,reference_start,reference_end,query_name,centered_query_start,centered_query_end,query_length,centered_position_type,centered_start,centered_end
33,chr1_1033080_+,chr1,1033080,+,GACCTACGGGGGCGGGTGTGGGGACGCCGGACTACGCGTCAGGAGT...,1008277,1033947,m54329U_210813_020940/24183559/ccs,-24816,861,25677,msp,-78,147
55,chr1_1033080_+,chr1,1033080,+,GACCTACGGGGGCGGGTGTGGGGACGCCGGACTACGCGTCAGGAGT...,1010739,1033452,m54329U_210813_020940/173737659/ccs,-22317,372,22689,msp,-42,58
152,chr1_1033080_+,chr1,1033080,+,CCTACGGGGGCGGGTGTGGGGACGCCGGAACTTACGCGTCAGGAGT...,1016995,1038612,m54329U_210323_190418/65799401/ccs,-18781,5567,24348,msp,-80,303
188,chr1_1033080_+,chr1,1033080,+,GACCTACGGGGGCGGGTGTGGGGACGCCGGACTACGCGTCAGGAGT...,1017128,1037380,m54329U_210814_130637/153158045/ccs,-15970,4300,20270,msp,-227,219
217,chr1_1033080_+,chr1,1033080,+,GACCTACGGGGGCGGGTGTGGGGACGCCGGACCTACGCGTCAGAGT...,1017682,1033494,m54329U_210323_190418/28969124/ccs,-15438,416,15854,msp,-71,220
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99788,chr1_15792178_+,chr1,15792178,+,AGGGGGACGGTCCCTGCCTAGGGTCCCCTGCGCAGCCTGGGAAGGC...,15777436,15796646,m54329U_210814_130637/115999192/ccs,-14771,4457,19228,msp,-67,99
99843,chr1_15792178_+,chr1,15792178,+,AGGGGGACGGTCCCTGCCTAGGGTCCCCTGCGCAGCCTGGGAAGGC...,15777763,15794222,m54329U_210323_190418/179176258/ccs,-14469,2035,16504,msp,-81,126
99896,chr1_15792178_+,chr1,15792178,+,AGGGGGACGGTCCCTGCCTAGGGTCCCCTGCGCAGCCTGGGAAGGC...,15778376,15806996,m64076_221119_202646/81527990/ccs,-13841,14819,28660,msp,-81,104
99943,chr1_15792178_+,chr1,15792178,+,AGGGGGACGGTCCCTGCCTAGGGTCCCCTGCGCAGCCTGGGAAGGC...,15779324,15792995,m54329U_210326_192251/126943836/ccs,-12896,806,13702,msp,-54,99


In [75]:
df[(df["centered_position_type"] == "msp") & (df["centered_start"] <= 0) & (df["centered_end"] >= motif_len)]

,motif_name,chrom,centering_position,strand,subset_sequence,reference_start,reference_end,query_name,centered_query_start,centered_query_end,query_length,centered_position_type,centered_start,centered_end
33,chr1_1033080_+,chr1,1033080,+,GACCTACGGGGGCGGGTGTGGGGACGCCGGACTACGCGTCAGGAGT...,1008277,1033947,m54329U_210813_020940/24183559/ccs,-24816,861,25677,msp,-78,147
55,chr1_1033080_+,chr1,1033080,+,GACCTACGGGGGCGGGTGTGGGGACGCCGGACTACGCGTCAGGAGT...,1010739,1033452,m54329U_210813_020940/173737659/ccs,-22317,372,22689,msp,-42,58
152,chr1_1033080_+,chr1,1033080,+,CCTACGGGGGCGGGTGTGGGGACGCCGGAACTTACGCGTCAGGAGT...,1016995,1038612,m54329U_210323_190418/65799401/ccs,-18781,5567,24348,msp,-80,303
188,chr1_1033080_+,chr1,1033080,+,GACCTACGGGGGCGGGTGTGGGGACGCCGGACTACGCGTCAGGAGT...,1017128,1037380,m54329U_210814_130637/153158045/ccs,-15970,4300,20270,msp,-227,219
217,chr1_1033080_+,chr1,1033080,+,GACCTACGGGGGCGGGTGTGGGGACGCCGGACCTACGCGTCAGAGT...,1017682,1033494,m54329U_210323_190418/28969124/ccs,-15438,416,15854,msp,-71,220
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99788,chr1_15792178_+,chr1,15792178,+,AGGGGGACGGTCCCTGCCTAGGGTCCCCTGCGCAGCCTGGGAAGGC...,15777436,15796646,m54329U_210814_130637/115999192/ccs,-14771,4457,19228,msp,-67,99
99843,chr1_15792178_+,chr1,15792178,+,AGGGGGACGGTCCCTGCCTAGGGTCCCCTGCGCAGCCTGGGAAGGC...,15777763,15794222,m54329U_210323_190418/179176258/ccs,-14469,2035,16504,msp,-81,126
99896,chr1_15792178_+,chr1,15792178,+,AGGGGGACGGTCCCTGCCTAGGGTCCCCTGCGCAGCCTGGGAAGGC...,15778376,15806996,m64076_221119_202646/81527990/ccs,-13841,14819,28660,msp,-81,104
99943,chr1_15792178_+,chr1,15792178,+,AGGGGGACGGTCCCTGCCTAGGGTCCCCTGCGCAGCCTGGGAAGGC...,15779324,15792995,m54329U_210326_192251/126943836/ccs,-12896,806,13702,msp,-54,99


In [72]:
df[df["centered_position_type"] == "msp"][["centered_start", "centered_end"]].describe()

,centered_start,centered_end
count,1995.000000,1995.000000
mean,-81.447619,124.157393
std,68.692121,69.302748
min,-1329.000000,-99.000000
25%,-94.000000,89.000000
50%,-66.000000,115.000000
75%,-45.000000,147.000000
max,0.000000,1556.000000


In [111]:
sum(df[["motif_name", "query_name"]].apply(tuple, 1).isin(msp_groups))

73671

In [109]:
# df[(df["centered_position_type"] == "msp") & (df["centered_end"] < 35)].shape[0]
73671 - 9

73662

In [89]:
# mask of MSP's containing a motif
motif_len = 35
msp_mask = (df["centered_position_type"] == "msp") & (df["centered_start"] <= 0) & (df["centered_end"] >= motif_len)
print("MSP's with a motif: " + "{:,}".format(msp_mask.sum()))
msp_groups = [(row["motif_name"], row["query_name"]) for idx, row in df[msp_mask].iterrows()]

# remove MSP's without a motif
print("Removing MSP's without a motif: " + "{:,}".format(
    (~np.logical_or(msp_mask, (df["centered_position_type"] == "m6a"))).sum()))
#df = df[np.logical_or(msp_mask, (df["centered_position_type"] == "m6a"))]

MSP's with a motif: 1,986
Removing MSP's without a motif: 1,316


In [123]:
len(msp_groups)

1986

In [90]:
[(row["motif_name"], row["query_name"]) for idx, row in df[msp_mask].iterrows()]

[('chr1_1033080_+', 'm54329U_210813_020940/24183559/ccs'),
 ('chr1_1033080_+', 'm54329U_210813_020940/173737659/ccs'),
 ('chr1_1033080_+', 'm54329U_210323_190418/65799401/ccs'),
 ('chr1_1033080_+', 'm54329U_210814_130637/153158045/ccs'),
 ('chr1_1033080_+', 'm54329U_210323_190418/28969124/ccs'),
 ('chr1_1033080_+', 'm54329U_210323_190418/92997266/ccs'),
 ('chr1_1033080_+', 'm54329U_210810_004956/66257203/ccs'),
 ('chr1_1033080_+', 'm54329U_210813_020940/5964715/ccs'),
 ('chr1_1033080_+', 'm64076_210328_012155/41027901/ccs'),
 ('chr1_1033080_+', 'm64076_210328_012155/58132989/ccs'),
 ('chr1_1033080_+', 'm54329U_210813_020940/85131496/ccs'),
 ('chr1_1033080_+', 'm64076_210328_012155/84345303/ccs'),
 ('chr1_1033080_+', 'm54329U_210323_190418/144837022/ccs'),
 ('chr1_1033080_+', 'm54329U_210810_004956/37945806/ccs'),
 ('chr1_1033080_+', 'm64076_210328_012155/39781822/ccs'),
 ('chr1_1033080_+', 'm54329U_210810_004956/137822662/ccs'),
 ('chr1_1033080_+', 'm54329U_210326_192251/158794909/ccs'

In [125]:
df[df[["motif_name", "query_name"]].apply(tuple, 1).isin(msp_groups)].shape

(73671, 14)

In [127]:
df.merge(pd.DataFrame(msp_groups, columns=["motif_name", "query_name"]))

AttributeError: 'DataFrame' object has no attribute 'msp'

In [74]:
# mask of MSP's containing a motif
motif_len = 35
msp_mask = (df["centered_position_type"] == "msp") & (df["centered_start"] <= 0) & (df["centered_end"] >= motif_len)
print("MSP's with a motif: " + "{:,}".format(msp_mask.sum()))
msp_groups = [(row["motif_name"], row["query_name"]) for idx, row in df[msp_mask].iterrows()]

# remove MSP's without a motif
print("Removing MSP's without a motif: " + "{:,}".format(
    (~np.logical_or(msp_mask, (df["centered_position_type"] == "m6a"))).sum()))
df = df[np.logical_or(msp_mask, (df["centered_position_type"] == "m6a"))]

# filter for rows with motifs within an MSP (gets both MSP's & m6a's)
df = df[df[["motif_name", "query_name"]].apply(tuple, 1).isin(msp_groups)]
print("Total observations from motifs within an MSP: " + "{:,}".format(df.shape[0]))

MSP's with a motif: 1,986
Removing MSP's without a motif: 1,316
Total observations from motifs within an MSP: 73,662


In [ ]:
# filter for rows with motifs within an MSP (gets both MSP's & m6a's)
df = df[df[["motif_name", "query_name"]].apply(tuple, 1).isin(msp_groups)]
print("Total observations from motifs within an MSP: " + "{:,}".format(df.shape[0]))

In [51]:
msp_mask.sum() + (df["centered_position_type"] == "m6a").sum()

89416

In [44]:
df[(df["query_name"] == "m64076_221119_202646/138085724/ccs")]

,motif_name,chrom,centering_position,strand,subset_sequence,reference_start,reference_end,query_name,centered_query_start,centered_query_end,query_length,centered_position_type,centered_start,centered_end
16603,chr1_3453274_+,chr1,3453274,+,TGTCCCCTCACTGGGCCTCAGTGACCCCACCGCACCTCAGTCCCCT...,3452462,3474434,m64076_221119_202646/138085724/ccs,-812,21148,21960,m6a,-100,-99
16604,chr1_3453274_+,chr1,3453274,+,TGTCCCCTCACTGGGCCTCAGTGACCCCACCGCACCTCAGTCCCCT...,3452462,3474434,m64076_221119_202646/138085724/ccs,-812,21148,21960,m6a,-5,-4
16605,chr1_3453274_+,chr1,3453274,+,TGTCCCCTCACTGGGCCTCAGTGACCCCACCGCACCTCAGTCCCCT...,3452462,3474434,m64076_221119_202646/138085724/ccs,-812,21148,21960,m6a,-2,-1
16606,chr1_3453274_+,chr1,3453274,+,TGTCCCCTCACTGGGCCTCAGTGACCCCACCGCACCTCAGTCCCCT...,3452462,3474434,m64076_221119_202646/138085724/ccs,-812,21148,21960,m6a,5,6
16607,chr1_3453274_+,chr1,3453274,+,TGTCCCCTCACTGGGCCTCAGTGACCCCACCGCACCTCAGTCCCCT...,3452462,3474434,m64076_221119_202646/138085724/ccs,-812,21148,21960,m6a,8,9
16608,chr1_3453274_+,chr1,3453274,+,TGTCCCCTCACTGGGCCTCAGTGACCCCACCGCACCTCAGTCCCCT...,3452462,3474434,m64076_221119_202646/138085724/ccs,-812,21148,21960,m6a,12,13
16609,chr1_3453274_+,chr1,3453274,+,TGTCCCCTCACTGGGCCTCAGTGACCCCACCGCACCTCAGTCCCCT...,3452462,3474434,m64076_221119_202646/138085724/ccs,-812,21148,21960,m6a,13,14
16610,chr1_3453274_+,chr1,3453274,+,TGTCCCCTCACTGGGCCTCAGTGACCCCACCGCACCTCAGTCCCCT...,3452462,3474434,m64076_221119_202646/138085724/ccs,-812,21148,21960,m6a,15,16
16611,chr1_3453274_+,chr1,3453274,+,TGTCCCCTCACTGGGCCTCAGTGACCCCACCGCACCTCAGTCCCCT...,3452462,3474434,m64076_221119_202646/138085724/ccs,-812,21148,21960,m6a,16,17
16612,chr1_3453274_+,chr1,3453274,+,TGTCCCCTCACTGGGCCTCAGTGACCCCACCGCACCTCAGTCCCCT...,3452462,3474434,m64076_221119_202646/138085724/ccs,-812,21148,21960,m6a,19,20


#### mask MSP's (current code)

##### positive data (merged)

In [58]:
# mask of MSP's containing a motif
motif_len = 35
msp_mask = (df["centered_position_type"] == "msp") & (df["centered_start"] <= 0) & (df["centered_end"] >= motif_len)
print("MSP's with a motif: " + "{:,}".format(msp_mask.sum()))
msp_groups = [(row["motif_name"], row["query_name"]) for idx, row in df[msp_mask].iterrows()]

# filter for rows with motifs within an MSP (gets both MSP's & m6a's)
df = df[df[["motif_name", "query_name"]].apply(tuple, 1).isin(msp_groups)]
print("Total observations from motifs within an MSP: " + "{:,}".format(df.shape[0]))

MSP's with a motif: 568,184
Total observations from motifs within an MSP: 23,097,432


In [59]:
# df with msp sizes
df_msp = df.loc[msp_mask, ["motif_name", "query_name", "centered_end", "centered_start"]]
df_msp["msp_size"] = df_msp["centered_end"] - df_msp["centered_start"]

# add msp size corresponding to each region
# match msp back to it's motif & fiber
df = df.merge(df_msp[["motif_name", "query_name", "msp_size"]], on=["motif_name", "query_name"])
print("Merged df shape w/ msp_size: " + "{:,}".format(df.shape[0]))

Merged df shape w/ msp_size: 23,097,577


In [60]:
# remove msp rows (already have msp size)
df = df[df["centered_position_type"] == "m6a"]
print("Total m6a observations: " + "{:,}".format(df.shape[0]))

# filter for m6a's within 40 bp range
m6a_range_mask = (df["centered_start"] >= -40) & (df["centered_end"] <= 75)
print("Total m6a's within motif & 40 bp flank: " + "{:,}".format(m6a_range_mask.sum()))
# apply mask
df = df[m6a_range_mask]

Total m6a observations: 22,525,795
Total m6a's within motif & 40 bp flank: 15,132,411


In [61]:
# group by motif & query name
grouping_cols = ["motif_name", "query_name"]
df_grouped = df.groupby(grouping_cols)

# get group names (keys)
group_names = list(df_grouped.groups.keys())
print("Unique motif-sequence groups: " + "{:,}".format(len(group_names)))

Unique motif-sequence groups: 568,182


##### positive data (L)

In [13]:
# mask of MSP's containing a motif
motif_len = 35
msp_mask = (df["centered_position_type"] == "msp") & (df["centered_start"] <= 0) & (df["centered_end"] >= motif_len)
print("MSP's with a motif: " + "{:,}".format(msp_mask.sum()))
msp_groups = [(row["motif_name"], row["query_name"]) for idx, row in df[msp_mask].iterrows()]

# filter for rows with motifs within an MSP (gets both MSP's & m6a's)
df = df[df[["motif_name", "query_name"]].apply(tuple, 1).isin(msp_groups)]
print("Total observations from motifs within an MSP: " + "{:,}".format(df.shape[0]))

MSP's with a motif: 238,348
Total observations from motifs within an MSP: 9,735,406


In [15]:
# df with msp sizes
df_msp = df.loc[msp_mask, ["motif_name", "query_name", "centered_end", "centered_start"]]
df_msp["msp_size"] = df_msp["centered_end"] - df_msp["centered_start"]

# add msp size corresponding to each region
# match msp back to it's motif & fiber
df = df.merge(df_msp[["motif_name", "query_name", "msp_size"]], on=["motif_name", "query_name"])
print("Merged df shape w/ msp_size: " + "{:,}".format(df.shape[0]))

Merged df shape w/ msp_size: 9,735,406


In [16]:
# remove msp rows (already have msp size)
df = df[df["centered_position_type"] == "m6a"]
print("Total m6a observations: " + "{:,}".format(df.shape[0]))

# filter for m6a's within 40 bp range
m6a_range_mask = (df["centered_start"] >= -40) & (df["centered_end"] <= 75)
print("Total m6a's within motif & 40 bp flank: " + "{:,}".format(m6a_range_mask.sum()))
# apply mask
df = df[m6a_range_mask]

Total m6a observations: 9,495,739
Total m6a's within motif & 40 bp flank: 6,388,435


In [19]:
# group by motif & query name
grouping_cols = ["motif_name", "query_name"]
df_grouped = df.groupby(grouping_cols)

# get group names (keys)
group_names = list(df_grouped.groups.keys())
print("Unique motif-sequence groups: " + "{:,}".format(len(group_names)))

Unique motif-sequence groups: 238,348


##### negative data (merged)

In [74]:
# mask of MSP's containing a motif
motif_len = 35
msp_mask = (df["centered_position_type"] == "msp") & (df["centered_start"] <= 0) & (df["centered_end"] >= motif_len)
print("MSP's with a motif: " + "{:,}".format(msp_mask.sum()))
msp_groups = [(row["motif_name"], row["query_name"]) for idx, row in df[msp_mask].iterrows()]

# filter for rows with motifs within an MSP (gets both MSP's & m6a's)
df = df[df[["motif_name", "query_name"]].apply(tuple, 1).isin(msp_groups)]
print("Total observations from motifs within an MSP: " + "{:,}".format(df.shape[0]))

MSP's with a motif: 217,575
Total observations from motifs within an MSP: 5,480,274


In [75]:
# df with msp sizes
df_msp = df.loc[msp_mask, ["motif_name", "query_name", "centered_end", "centered_start"]]
df_msp["msp_size"] = df_msp["centered_end"] - df_msp["centered_start"]

# add msp size corresponding to each region
# match msp back to it's motif & fiber
df = df.merge(df_msp[["motif_name", "query_name", "msp_size"]], on=["motif_name", "query_name"])
print("Merged df shape w/ msp_size: " + "{:,}".format(df.shape[0]))

Merged df shape w/ msp_size: 5,481,244


In [76]:
# remove msp rows (already have msp size)
df = df[df["centered_position_type"] == "m6a"]
print("Total m6a observations: " + "{:,}".format(df.shape[0]))

# filter for m6a's within 40 bp range
m6a_range_mask = (df["centered_start"] >= -40) & (df["centered_end"] <= 75)
print("Total m6a's within motif & 40 bp flank: " + "{:,}".format(m6a_range_mask.sum()))
# apply mask
df = df[m6a_range_mask]

Total m6a observations: 5,256,334
Total m6a's within motif & 40 bp flank: 4,323,830


In [77]:
# group by motif & query name
grouping_cols = ["motif_name", "query_name"]
df_grouped = df.groupby(grouping_cols)

# get group names (keys)
group_names = list(df_grouped.groups.keys())
print("Unique motif-sequence groups: " + "{:,}".format(len(group_names)))

Unique motif-sequence groups: 217,557


##### negative data (L)

In [78]:
def filt_msps(df, motif_len=35):
    '''Filters for motif/query instances with a motif within 
    a MSP and adds MSP length to each motif/query group.'''
    
    # position of MSPs containing a motif
    msp_mask = (df["centered_position_type"] == "msp") & (df["centered_start"] <= 0) & (df["centered_end"] >= motif_len)
    print("MSP's with a motif: " + "{:,}".format(msp_mask.sum()))
    msp_groups = [(row["motif_name"], row["query_name"]) for idx, row in df[msp_mask].iterrows()]
    
    # filter for rows with motifs within an MSP (gets both MSP's & m6a's)
    df = df[df[["motif_name", "query_name"]].apply(tuple, 1).isin(msp_groups)]
    print("Total observations from motifs within an MSP: " + "{:,}".format(df.shape[0]))
    
    # add MSP size corresponding to each group
    # df with msp sizes
    df_msp = df.loc[msp_mask, ["motif_name", "query_name", "centered_end", "centered_start"]]
    df_msp["msp_size"] = df_msp["centered_end"] - df_msp["centered_start"]

    # match msp back to it's motif & fiber
    df = df.merge(df_msp[["motif_name", "query_name", "msp_size"]], on=["motif_name", "query_name"])
    print("Merged df shape w/ msp_size: " + "{:,}".format(df.shape[0]))
    return df

In [79]:
# filter motif/query groups for those with an MSP containing a motif & add MSP size.
df = filt_msps(df)

MSP's with a motif: 65,952
Total observations from motifs within an MSP: 1,663,294
Merged df shape w/ msp_size: 1,663,752


In [80]:
# remove MSP rows (already have msp size)
df = df[df["centered_position_type"] == "m6a"]
print("Total m6a observations: " + "{:,}".format(df.shape[0]))

# filter for m6a's within 40 bp range
m6a_range_mask = (df["centered_start"] >= -40) & (df["centered_end"] <= 75)
print("Total m6a's within 40 bp flank of motif: " + "{:,}".format(m6a_range_mask.sum()))
# apply mask
df = df[m6a_range_mask]

Total m6a observations: 1,595,563
Total m6a's within 40 bp flank of motif: 1,325,857


In [81]:
# group by motif & query name
grouping_cols = ["motif_name", "query_name"]
df_grouped = df.groupby(grouping_cols)

# get group names (keys)
group_names = list(df_grouped.groups.keys())
print("Unique motif-sequence groups: " + "{:,}".format(len(group_names)))

Unique motif-sequence groups: 65,943


### extract features by motif-query group

potential features
* [x] read length [query_length]
* [x] 3-mer k-mer count within canonical motif (exclude k-mers without AT)
* [x] m6a count within each k-mer (m6a/total ATs)
* [x] m6a count for: (motif, 40 bp flank left, 40 bp flank right)
* [x] AT count for: (motif, flank left, flank right)
* [x] proportion of bases that are AT per element (0-1)
* [x] proportion of methylated ATs per element (motif, flank left, flank right) (0-1)
> $$ m6a\ prop = m6a\ count / AT\ count $$
> &uarr; proportion of ATs methylated (m6a count/AT count) or proportion of methylated ATs over the region (m6a count/region length)?
* [x] total methylation across the read (% of ATs that were methylated across read)
* [x] MSP size
* [ ] do a run length encoding of motif in AT space

In [87]:
def get_motif_seq(subset_sequence):
    '''Get motif sequence from subset_sequence.'''
    center_idx = 100
    motif_len = 35
    motif_seq = subset_sequence[center_idx:(center_idx+motif_len)]
    return motif_seq

In [83]:
def get_kmers(seq, k):
    '''Decompose the input sequence (str) into k-mers (str).'''
    num_kmers = len(seq) - k + 1
    seqs = [seq[i:i+k] for i in range(num_kmers)]
    return seqs

In [84]:
def agg_features(x, feature_cols):
    '''Collects features. Returns a df with feature info per unique group.'''
    feature_cols = ["msp_size",
                    "left_m6a_count", "right_m6a_count", "motif_m6a_count", # m6a count
                    "left_AT_count", "right_AT_count", "motif_AT_count", # AT count
                    "left_AT_prop", "right_AT_prop", "motif_AT_prop", # proportion of bases that are AT
                    "left_m6a_prop", "right_m6a_prop", "motif_m6a_prop", # proportion of ATs that are methylated
                    "total_m6a_prop", # proportion of ATs that are methylated in subseq_sequence
                   ] + feature_cols
    
    # dict to hold info
    d = dict((col, 0) for col in feature_cols)
    
    # msp size
    d[feature_cols[0]] = x["msp_size"].unique()[0]
    
    # m6a count flanking left, right, motif
    d[feature_cols[1]] = (x["centered_end"] < 0).sum()
    d[feature_cols[2]] = (x["centered_start"] >= 35).sum()
    d[feature_cols[3]] = ((x["centered_start"] >= 0) & (x["centered_end"] <= 35)).sum()
    
    # sequences
    center = 100
    motif_len = 35
    subseq = x["subset_sequence"].unique()[0]
    subseq_motif = subseq[center:(center+motif_len)]
    # length of flank in bp
    flank_len = 40
    # flank left & flank_right
    subseq_l = subseq[center-flank_len:center]
    subseq_r = subseq[center+motif_len:center+motif_len+flank_len]
    
    # AT count
    d[feature_cols[4]] = (subseq_l.count("A") + subseq_l.count("T"))
    d[feature_cols[5]] = (subseq_r.count("A") + subseq_r.count("T"))
    d[feature_cols[6]] = (subseq_motif.count("A") + subseq_motif.count("T"))
    
    # proportion of bases that are AT
    d[feature_cols[7]] = (subseq_l.count("A") + subseq_l.count("T"))/flank_len
    d[feature_cols[8]] = (subseq_r.count("A") + subseq_r.count("T"))/flank_len
    d[feature_cols[9]] = (subseq_motif.count("A") + subseq_motif.count("T"))/motif_len
    
    # proportion of methylated ATs (m6a_count/AT_count)
    d[feature_cols[10]] = d[feature_cols[1]]/d[feature_cols[4]]
    d[feature_cols[11]] = d[feature_cols[2]]/d[feature_cols[5]]
    d[feature_cols[12]] = d[feature_cols[3]]/d[feature_cols[6]]
    d[feature_cols[13]] = (x["centered_position_type"] == "m6a").sum()/(subseq.count("A") + subseq.count("T"))
    
    
    #----- k-mer info -----#
    
    # m6a bool
    m6a_bool = x[(x["centered_start"] >= 0) & (x["centered_end"] <= 35)]["centered_start"].values
    m6a_bool = [1 if (i in m6a_bool) else 0 for i in range(0, 35)]
    
    # decompose motif seq into k-mers
    motif_kmers = get_kmers(subseq_motif, 3)
    m6a_kmers = get_kmers(m6a_bool, 3)
    
    # make bool of ATs in motif seq
    motifis_AT = [("A" in kmer or "T" in kmer) for kmer in motif_kmers]
    
    # drop motif instances without an AT
    motif_kmers = list(np.array(motif_kmers)[motifis_AT])
    m6a_kmers = list(np.array(m6a_kmers)[motifis_AT])
    
    # make kmer_dict
    kmer_info = defaultdict(lambda: defaultdict(int))
    # collect count & m6a count per k-mer
    for kmer, m6as in zip(motif_kmers, m6a_kmers):
        kmer_info[kmer]["count"] += 1
        kmer_info[kmer]["m6a"] += sum(m6as)
    
    # get kmer_m6a proportion
    for kmer, v in kmer_info.items():
        d[kmer+"_count"] = v["count"]
        d[kmer+"_m6a_prop"] = v["m6a"]/((Counter(kmer)["T"] + Counter(kmer)["A"]) * v["count"])
    
    return pd.Series(d, index=list(d.keys()))

##### positive data (merged)

In [62]:
print("Total rows: " + "{:,}".format(df.shape[0]))
print("Unique motif-sequence groups: " + "{:,}".format(len(group_names)))
print("Unique motifs: " + "{:,}".format(df["motif_name"].nunique()))
print("Unique query_names: " + "{:,}".format(df["query_name"].nunique()))

Total rows: 15,132,411
Unique motif-sequence groups: 568,182
Unique motifs: 15,761
Unique query_names: 446,256


In [63]:
%%time

# get all possible k-mers
all_kmers = []
for seq in df["subset_sequence"]:
    all_kmers.extend(get_kmers(get_motif_seq(seq), 3))

# sorted unique k-mers containing AT
all_kmers = [*set(all_kmers)]
all_kmers = sorted([kmer for kmer in all_kmers if ("T" in kmer) or ("A" in kmer)])

# make k-mer columns
kmer_cols = list(itertools.chain(*[[kmer+"_count", kmer+"_m6a_prop"] for kmer in all_kmers]))

CPU times: user 1min 9s, sys: 14.2 s, total: 1min 23s
Wall time: 1min 23s


In [64]:
%%time
# aggregate features
res = df.groupby(grouping_cols).apply(lambda x: agg_features(x, kmer_cols)).reset_index()
# change col dtypes for memory
res[res.columns.tolist()[2:]] = res[res.columns.tolist()[2:]].apply(pd.to_numeric, downcast="float")
print("Total rows: " + "{:,}".format(res.shape[0]))
print("Total columns: " + "{:,}".format(res.shape[1]))

/tmp/ipykernel_60500/2521446196.py:45: RuntimeWarning: invalid value encountered in long_scalars
  d[feature_cols[11]] = d[feature_cols[2]]/d[feature_cols[5]]


Total rows: 568,182
Total columns: 128
CPU times: user 16min 2s, sys: 9.84 s, total: 16min 12s
Wall time: 16min 17s


In [66]:
res.head()

,motif_name,query_name,msp_size,left_m6a_count,right_m6a_count,motif_m6a_count,left_AT_count,right_AT_count,motif_AT_count,left_AT_prop,...,TGT_count,TGT_m6a_prop,TTA_count,TTA_m6a_prop,TTC_count,TTC_m6a_prop,TTG_count,TTG_m6a_prop,TTT_count,TTT_m6a_prop
0,chr10_100046241_-,m54329U_210323_190418/105513905/ccs,174.0,13.0,10.0,0.0,21.0,16.0,11.0,0.525,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,chr10_100046241_-,m54329U_210323_190418/107414595/ccs,196.0,14.0,14.0,2.0,21.0,16.0,11.0,0.525,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,chr10_100046241_-,m54329U_210323_190418/117571949/ccs,155.0,15.0,9.0,2.0,21.0,17.0,11.0,0.525,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,chr10_100046241_-,m54329U_210323_190418/135595143/ccs,161.0,15.0,13.0,3.0,21.0,17.0,11.0,0.525,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,chr10_100046241_-,m54329U_210323_190418/152242932/ccs,98.0,11.0,14.0,10.0,21.0,17.0,11.0,0.525,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


##### positive data (L)

In [21]:
print("Total rows: " + "{:,}".format(df.shape[0]))
print("Unique motif-sequence groups: " + "{:,}".format(len(group_names)))
print("Unique motifs: " + "{:,}".format(df["motif_name"].nunique()))
print("Unique query_names: " + "{:,}".format(df["query_name"].nunique()))

Total rows: 6,388,435
Unique motif-sequence groups: 238,348
Unique motifs: 6,504
Unique query_names: 223,211


In [22]:
print("{:,}".format(47836/6388435*100))

0.7487905879922079


In [28]:
%%time

# get all possible k-mers
all_kmers = []
for seq in df["subset_sequence"]:
    all_kmers.extend(get_kmers(get_motif_seq(seq), 3))

# sorted unique k-mers containing AT
all_kmers = [*set(all_kmers)]
all_kmers = sorted([kmer for kmer in all_kmers if ("T" in kmer) or ("A" in kmer)])

# make k-mer columns
kmer_cols = list(itertools.chain(*[[kmer+"_count", kmer+"_m6a_prop"] for kmer in all_kmers]))

CPU times: user 28.9 s, sys: 5.43 s, total: 34.3 s
Wall time: 34.4 s


In [29]:
%%time
# aggregate features
res = df.groupby(grouping_cols).apply(lambda x: agg_features(x, kmer_cols)).reset_index()
# change col dtypes for memory
res[res.columns.tolist()[2:]] = res[res.columns.tolist()[2:]].apply(pd.to_numeric, downcast="float")
print("Total rows: " + "{:,}".format(res.shape[0]))
print("Total columns: " + "{:,}".format(res.shape[1]))

Total rows: 238,348
Total columns: 128
CPU times: user 6min 47s, sys: 3.7 s, total: 6min 51s
Wall time: 6min 53s


In [31]:
res.head()

,motif_name,query_name,msp_size,left_m6a_count,right_m6a_count,motif_m6a_count,left_AT_count,right_AT_count,motif_AT_count,left_AT_prop,...,TGT_count,TGT_m6a_prop,TTA_count,TTA_m6a_prop,TTC_count,TTC_m6a_prop,TTG_count,TTG_m6a_prop,TTT_count,TTT_m6a_prop
0,chr10_100338605_+,m54329U_210323_190418/12585455/ccs,200.0,13.0,13.0,3.0,18.0,18.0,13.0,0.45,...,0.0,0.0,1.0,0.000000,0.0,0.0,0.0,0.0,1.0,0.333333
1,chr10_100338605_+,m54329U_210323_190418/16515087/ccs,212.0,12.0,12.0,4.0,18.0,18.0,13.0,0.45,...,0.0,0.0,1.0,0.333333,0.0,0.0,0.0,0.0,1.0,0.666667
2,chr10_100338605_+,m54329U_210323_190418/169477611/ccs,181.0,10.0,12.0,2.0,18.0,18.0,13.0,0.45,...,0.0,0.0,1.0,0.000000,0.0,0.0,0.0,0.0,1.0,0.000000
3,chr10_100338605_+,m54329U_210323_190418/173999297/ccs,134.0,9.0,11.0,8.0,18.0,18.0,13.0,0.45,...,0.0,0.0,1.0,0.333333,0.0,0.0,0.0,0.0,1.0,0.666667
4,chr10_100338605_+,m54329U_210323_190418/180160021/ccs,194.0,12.0,12.0,1.0,18.0,18.0,13.0,0.45,...,0.0,0.0,1.0,0.000000,0.0,0.0,0.0,0.0,1.0,0.333333


##### negative data (merged)

In [78]:
print("Total rows: " + "{:,}".format(df.shape[0]))
print("Unique motif-sequence groups: " + "{:,}".format(len(group_names)))
print("Unique motifs: " + "{:,}".format(df["motif_name"].nunique()))
print("Unique query_names: " + "{:,}".format(df["query_name"].nunique()))

Total rows: 4,323,830
Unique motif-sequence groups: 217,557
Unique motifs: 203,570
Unique query_names: 65,867


In [79]:
%%time

# get all possible k-mers
all_kmers = []
for seq in df["subset_sequence"]:
    all_kmers.extend(get_kmers(get_motif_seq(seq), 3))

# sorted unique k-mers containing AT
all_kmers = [*set(all_kmers)]
all_kmers = sorted([kmer for kmer in all_kmers if ("T" in kmer) or ("A" in kmer)])

# make k-mer columns
kmer_cols = list(itertools.chain(*[[kmer+"_count", kmer+"_m6a_prop"] for kmer in all_kmers]))

CPU times: user 20.1 s, sys: 2.57 s, total: 22.6 s
Wall time: 22.7 s


In [80]:
%%time
# aggregate features
res = df.groupby(grouping_cols).apply(lambda x: agg_features(x, kmer_cols)).reset_index()
# change col dtypes for memory
res[res.columns.tolist()[2:]] = res[res.columns.tolist()[2:]].apply(pd.to_numeric, downcast="float")
print("Total rows: " + "{:,}".format(res.shape[0]))
print("Total columns: " + "{:,}".format(res.shape[1]))

/tmp/ipykernel_60500/2521446196.py:46: RuntimeWarning: invalid value encountered in long_scalars
  d[feature_cols[12]] = d[feature_cols[3]]/d[feature_cols[6]]
/tmp/ipykernel_60500/2521446196.py:46: RuntimeWarning: invalid value encountered in long_scalars
  d[feature_cols[12]] = d[feature_cols[3]]/d[feature_cols[6]]
/tmp/ipykernel_60500/2521446196.py:46: RuntimeWarning: invalid value encountered in long_scalars
  d[feature_cols[12]] = d[feature_cols[3]]/d[feature_cols[6]]
/tmp/ipykernel_60500/2521446196.py:46: RuntimeWarning: invalid value encountered in long_scalars
  d[feature_cols[12]] = d[feature_cols[3]]/d[feature_cols[6]]
/tmp/ipykernel_60500/2521446196.py:46: RuntimeWarning: invalid value encountered in long_scalars
  d[feature_cols[12]] = d[feature_cols[3]]/d[feature_cols[6]]
/tmp/ipykernel_60500/2521446196.py:46: RuntimeWarning: invalid value encountered in long_scalars
  d[feature_cols[12]] = d[feature_cols[3]]/d[feature_cols[6]]
/tmp/ipykernel_60500/2521446196.py:46: Runtime

Total rows: 217,557
Total columns: 128
CPU times: user 6min 8s, sys: 951 ms, total: 6min 9s
Wall time: 6min 11s


In [81]:
res.head()

,motif_name,query_name,msp_size,left_m6a_count,right_m6a_count,motif_m6a_count,left_AT_count,right_AT_count,motif_AT_count,left_AT_prop,...,TGT_count,TGT_m6a_prop,TTA_count,TTA_m6a_prop,TTC_count,TTC_m6a_prop,TTG_count,TTG_m6a_prop,TTT_count,TTT_m6a_prop
0,chr10_100001134_+,m54329U_210326_192251/136053044/ccs,154.0,6.0,0.0,1.0,16.0,14.0,16.0,0.400,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
1,chr10_100007458_-,m54329U_210813_020940/108857329/ccs,97.0,1.0,3.0,8.0,20.0,14.0,11.0,0.500,...,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,chr10_100009645_+,m54329U_210813_020940/108857329/ccs,89.0,3.0,4.0,7.0,9.0,11.0,9.0,0.225,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,chr10_100009901_+,m54329U_210813_020940/108857329/ccs,268.0,6.0,6.0,4.0,9.0,10.0,7.0,0.225,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,chr10_100012002_-,m54329U_210813_020940/108857329/ccs,69.0,3.0,3.0,9.0,18.0,22.0,12.0,0.450,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


##### negative data (L)

In [85]:
print("Total rows: " + "{:,}".format(df.shape[0]))
print("Unique motif-sequence groups: " + "{:,}".format(len(group_names)))
print("Unique motifs: " + "{:,}".format(df["motif_name"].nunique()))
print("Unique query_names: " + "{:,}".format(df["query_name"].nunique()))

Total rows: 1,325,857
Unique motif-sequence groups: 65,943
Unique motifs: 61,923
Unique query_names: 38,675


In [90]:
%%time

# get all possible k-mers
all_kmers = []
for seq in df["subset_sequence"]:
    all_kmers.extend(get_kmers(get_motif_seq(seq), 3))

# sorted unique k-mers containing AT
all_kmers = [*set(all_kmers)]
all_kmers = sorted([kmer for kmer in all_kmers if ("T" in kmer) or ("A" in kmer)])

# make k-mer columns
kmer_cols = list(itertools.chain(*[[kmer+"_count", kmer+"_m6a_prop"] for kmer in all_kmers]))

CPU times: user 6.1 s, sys: 1.07 s, total: 7.17 s
Wall time: 7.19 s


In [104]:
%%time
# aggregate features
res = df.groupby(grouping_cols).apply(lambda x: agg_features(x, kmer_cols)).reset_index()

CPU times: user 1min 57s, sys: 553 ms, total: 1min 58s
Wall time: 1min 58s


In [116]:
print("{:,}".format(sys.getsizeof(res)))
print("{:,}".format(res.memory_usage().sum()))
res.info()

77,271,979
67,525,760
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65943 entries, 0 to 65942
Columns: 128 entries, motif_name to TTT_m6a_prop
dtypes: float64(126), object(2)
memory usage: 64.4+ MB


In [117]:
# change col dtypes for memory
res[res.columns.tolist()[2:]] = res[res.columns.tolist()[2:]].apply(
    pd.to_numeric, downcast="float")

print("{:,}".format(sys.getsizeof(res)))
print("{:,}".format(res.memory_usage().sum()))
res.info()

44,036,707
34,290,488
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65943 entries, 0 to 65942
Columns: 128 entries, motif_name to TTT_m6a_prop
dtypes: float32(126), object(2)
memory usage: 32.7+ MB


In [91]:
%%time
# aggregate features
res = df.groupby(grouping_cols).apply(lambda x: agg_features(x, kmer_cols)).reset_index()
# change col dtypes for memory
res[res.columns.tolist()[2:]] = res[res.columns.tolist()[2:]].apply(pd.to_numeric, downcast="float")
print("Total rows: " + "{:,}".format(res.shape[0]))
print("Total columns: " + "{:,}".format(res.shape[1]))

Total rows: 65,943
Total columns: 128
CPU times: user 1min 53s, sys: 1.12 s, total: 1min 54s
Wall time: 1min 55s


In [93]:
res.head()

,motif_name,query_name,msp_size,left_m6a_count,right_m6a_count,motif_m6a_count,left_AT_count,right_AT_count,motif_AT_count,left_AT_prop,...,TGT_count,TGT_m6a_prop,TTA_count,TTA_m6a_prop,TTC_count,TTC_m6a_prop,TTG_count,TTG_m6a_prop,TTT_count,TTT_m6a_prop
0,chr10_100001134_+,m54329U_210326_192251/136053044/ccs,154.0,6.0,0.0,1.0,16.0,14.0,16.0,0.400,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.000000
1,chr10_100009901_+,m54329U_210813_020940/108857329/ccs,268.0,6.0,6.0,4.0,9.0,10.0,7.0,0.225,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,chr10_100076460_+,m64076_210328_012155/112330052/ccs,69.0,1.0,10.0,8.0,22.0,22.0,15.0,0.550,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,chr10_100085646_-,m54329U_210810_004956/12124794/ccs,55.0,2.0,3.0,5.0,22.0,23.0,15.0,0.550,...,0.0,0.0,1.0,0.0,1.0,0.5,1.0,0.5,1.0,0.333333
4,chr10_100118172_-,m64076_221119_202646/24772945/ccs,71.0,2.0,6.0,3.0,30.0,25.0,16.0,0.750,...,1.0,0.5,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.000000


In [95]:
make_IDs(res, col_name="motif_query")

/tmp/ipykernel_88631/1120863114.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.insert(loc=0, column=col_name, value=df["motif_name"]+"/"+df["query_name"])


In [96]:
res.head()

,motif_query,motif_name,query_name,msp_size,left_m6a_count,right_m6a_count,motif_m6a_count,left_AT_count,right_AT_count,motif_AT_count,...,TGT_count,TGT_m6a_prop,TTA_count,TTA_m6a_prop,TTC_count,TTC_m6a_prop,TTG_count,TTG_m6a_prop,TTT_count,TTT_m6a_prop
0,chr10_100001134_+/m54329U_210326_192251/136053...,chr10_100001134_+,m54329U_210326_192251/136053044/ccs,154.0,6.0,0.0,1.0,16.0,14.0,16.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.000000
1,chr10_100009901_+/m54329U_210813_020940/108857...,chr10_100009901_+,m54329U_210813_020940/108857329/ccs,268.0,6.0,6.0,4.0,9.0,10.0,7.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,chr10_100076460_+/m64076_210328_012155/1123300...,chr10_100076460_+,m64076_210328_012155/112330052/ccs,69.0,1.0,10.0,8.0,22.0,22.0,15.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,chr10_100085646_-/m54329U_210810_004956/121247...,chr10_100085646_-,m54329U_210810_004956/12124794/ccs,55.0,2.0,3.0,5.0,22.0,23.0,15.0,...,0.0,0.0,1.0,0.0,1.0,0.5,1.0,0.5,1.0,0.333333
4,chr10_100118172_-/m64076_221119_202646/2477294...,chr10_100118172_-,m64076_221119_202646/24772945/ccs,71.0,2.0,6.0,3.0,30.0,25.0,16.0,...,1.0,0.5,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.000000


In [47]:
%%time
# aggregate features
res = df.groupby(grouping_cols).apply(lambda x: agg_features(x, kmer_cols)).reset_index()
# change col dtypes for memory
res[res.columns.tolist()[2:]] = res[res.columns.tolist()[2:]].apply(pd.to_numeric, downcast="float")
print("Total rows: " + "{:,}".format(res.shape[0]))
print("Total columns: " + "{:,}".format(res.shape[1]))

Total rows: 65,943
Total columns: 128
CPU times: user 1min 53s, sys: 1.29 s, total: 1min 54s
Wall time: 1min 55s


In [49]:
res.head()

,motif_name,query_name,msp_size,left_m6a_count,right_m6a_count,motif_m6a_count,left_AT_count,right_AT_count,motif_AT_count,left_AT_prop,...,TGT_count,TGT_m6a_prop,TTA_count,TTA_m6a_prop,TTC_count,TTC_m6a_prop,TTG_count,TTG_m6a_prop,TTT_count,TTT_m6a_prop
0,chr10_100001134_+,m54329U_210326_192251/136053044/ccs,154.0,6.0,0.0,1.0,16.0,14.0,16.0,0.400,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.000000
1,chr10_100009901_+,m54329U_210813_020940/108857329/ccs,268.0,6.0,6.0,4.0,9.0,10.0,7.0,0.225,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,chr10_100076460_+,m64076_210328_012155/112330052/ccs,69.0,1.0,10.0,8.0,22.0,22.0,15.0,0.550,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,chr10_100085646_-,m54329U_210810_004956/12124794/ccs,55.0,2.0,3.0,5.0,22.0,23.0,15.0,0.550,...,0.0,0.0,1.0,0.0,1.0,0.5,1.0,0.5,1.0,0.333333
4,chr10_100118172_-,m64076_221119_202646/24772945/ccs,71.0,2.0,6.0,3.0,30.0,25.0,16.0,0.750,...,1.0,0.5,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.000000


#### run length encoding Mitchell example

In [148]:
m6a = [0,0,0,1,1,1,0,0,0,0,0,0,1]
motif_m6a = m6a[100:135]
seq = "AAGGCCTTAAGGGAAAA"*20
motif_seq = seq[100:135]
# AT mask
motifis_AT = [(base == "A" or base == "T") for base in motif_seq]

# take motif m6a & subset by positions that are actually AT
np.array(motif_m6a)[np.array(motifis_AT)]

IndexError: boolean index did not match indexed array along dimension 0; dimension is 0 but corresponding boolean dimension is 35

In [149]:
m6a = [0,0,0,1,1,1,0,0,0,0,0,0,1]
test_rle = np.array(m6a)

In [ ]:
# numpy run length encoding Python (could do in a for loop but there's probably a 1 liner)
# see stack overflow link Mitchell sent
the function in the first response looks like a reasonable way to code it up to him

### Save feature info

In [84]:
# set folders
project_dir = "/mmfs1/gscratch/stergachislab/mwperez/ctcf-footprinting"
output_dir = "{}/feature_data".format(project_dir)

##### positive data (merged)

In [68]:
# save POSITIVE merged feature info to txt
motif_type = "merged"
output_file = "{}/CTCF_m6a_fiberseq_{}_100bp_features.txt".format(output_dir, motif_type)
print("Saving to output file: {}".format(output_file))
res.to_csv(output_file, header=True, index=None, sep="\t",)

Saving to output file: /mmfs1/gscratch/stergachislab/mwperez/ctcf-footprinting/candidate_footprints/CTCF_m6a_fiberseq_merged_100bp_features.txt


##### positive data (L)

In [34]:
# save POSITIVE L feature info to txt
motif_type = "L"
output_file = "{}/CTCF_m6a_fiberseq_{}_100bp_features.txt".format(output_dir, motif_type)
print("Saving to output file: {}".format(output_file))
res.to_csv(output_file, header=True, index=None, sep="\t",)

Saving to output file: /mmfs1/gscratch/stergachislab/mwperez/ctcf-footprinting/candidate_footprints/CTCF_m6a_fiberseq_L_100bp_features.txt


##### negative data (merged)

In [85]:
# save POSITIVE merged feature info to txt
motif_type = "merged"
output_file = "{}/CTCF_negative_m6a_fiberseq_{}_100bp_small_features.txt".format(output_dir, motif_type)
print("Saving to output file: {}".format(output_file))
res.to_csv(output_file, header=True, index=None, sep="\t",)

Saving to output file: /mmfs1/gscratch/stergachislab/mwperez/ctcf-footprinting/feature_data/CTCF_negative_m6a_fiberseq_merged_100bp_small_features.txt


##### negative data (L)

In [51]:
# save NEGATIVE L feature info to txt
motif_type = "L"
output_file = "{}/CTCF_negative_m6a_fiberseq_{}_100bp_small_features.txt".format(output_dir, motif_type)
print("Saving to output file: {}".format(output_file))
res.to_csv(output_file, header=True, index=None, sep="\t",)

Saving to output file: /mmfs1/gscratch/stergachislab/mwperez/ctcf-footprinting/candidate_footprints/CTCF_negative_m6a_fiberseq_L_100bp_small_features.txt


In [97]:
data_file

'/mmfs1/gscratch/stergachislab/mwperez/ctcf-footprinting/candidate_footprints/CTCF_negative_m6a_fiberseq_L_100bp_small.txt'

In [102]:
output_dir

'/mmfs1/gscratch/stergachislab/mwperez/ctcf-footprinting/feature_data'

In [103]:
"{}/{}".format(output_dir, os.path.basename(data_file).replace(".txt", "_features.txt"))

'/mmfs1/gscratch/stergachislab/mwperez/ctcf-footprinting/feature_data/CTCF_negative_m6a_fiberseq_L_100bp_small_features.txt'

## Format output to pin

### balancing positive & negative data

In [4]:
# set folders
project_dir = "/mmfs1/gscratch/stergachislab/mwperez/ctcf-footprinting"
data_dir = "{}/candidate_footprints".format(project_dir)
output_dir = "{}/feature_data".format(project_dir)

In [7]:
# CTCF merged 5% negative
data_file = "{}/CTCF_m6a_fiberseq_merged_100bp_features-small_5_noNaN.pin".format(output_dir)

In [8]:
%%time
# read in data
df = pd.read_csv(data_file, sep="\t")
print(f"{df.shape[0]:,d}")

1,902,730
CPU times: user 12.2 s, sys: 1.96 s, total: 14.2 s
Wall time: 14.3 s


In [15]:
df.Label.value_counts()[1]

814274

In [13]:
df.head()

,SpecID,Label,msp_size,left_m6a_count,right_m6a_count,motif_m6a_count,left_AT_count,right_AT_count,motif_AT_count,left_AT_prop,...,TTA_m6a_prop,TTC_count,TTC_m6a_prop,TTG_count,TTG_m6a_prop,TTT_count,TTT_m6a_prop,Peptide,Proteins,scannr
0,chr10_100046241_-/m54329U_210323_190418/105513...,1,174.0,13.0,10.0,0.0,21.0,16.0,11.0,0.525,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0
1,chr10_100046241_-/m54329U_210323_190418/107414...,1,196.0,14.0,14.0,2.0,21.0,16.0,11.0,0.525,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,1
2,chr10_100046241_-/m54329U_210323_190418/117571...,1,155.0,15.0,9.0,2.0,21.0,17.0,11.0,0.525,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,2,2
3,chr10_100046241_-/m54329U_210323_190418/135595...,1,161.0,15.0,13.0,3.0,21.0,17.0,11.0,0.525,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,3,3
4,chr10_100046241_-/m54329U_210323_190418/152242...,1,98.0,11.0,14.0,10.0,21.0,17.0,11.0,0.525,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,4,4


In [14]:
# balance data
df_neg = df[df["Label"] == -1]
df_neg.shape

(1088456, 131)

In [16]:
# get the same number of rows as positive data
neg_sample = df_neg.sample(n=df.Label.value_counts()[1])

In [18]:
neg_sample = neg_sample.reset_index(drop=True)

In [20]:
# merge positive & subsampled negative data
d = pd.concat([df[df["Label"] == 1], neg_sample], axis=0)
d.shape

(1628548, 131)

In [22]:
# Percolator tab-delimited format
pin_file = "{}/CTCF_m6a_fiberseq_merged_100bp_features-small_5_noNaN_balanced.pin".format(output_dir)
d.to_csv(pin_file, sep="\t", header=True, index=False)
print("Saved pin file to: {}".format(pin_file))

Saved pin file to: /mmfs1/gscratch/stergachislab/mwperez/ctcf-footprinting/feature_data/CTCF_m6a_fiberseq_merged_100bp_features-small_5_noNaN_balanced.pin


In [23]:
d

,SpecID,Label,msp_size,left_m6a_count,right_m6a_count,motif_m6a_count,left_AT_count,right_AT_count,motif_AT_count,left_AT_prop,...,TTA_m6a_prop,TTC_count,TTC_m6a_prop,TTG_count,TTG_m6a_prop,TTT_count,TTT_m6a_prop,Peptide,Proteins,scannr
0,chr10_100046241_-/m54329U_210323_190418/105513...,1,174.0,13.0,10.0,0.0,21.0,16.0,11.0,0.525,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0
1,chr10_100046241_-/m54329U_210323_190418/107414...,1,196.0,14.0,14.0,2.0,21.0,16.0,11.0,0.525,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,1
2,chr10_100046241_-/m54329U_210323_190418/117571...,1,155.0,15.0,9.0,2.0,21.0,17.0,11.0,0.525,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,2,2
3,chr10_100046241_-/m54329U_210323_190418/135595...,1,161.0,15.0,13.0,3.0,21.0,17.0,11.0,0.525,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,3,3
4,chr10_100046241_-/m54329U_210323_190418/152242...,1,98.0,11.0,14.0,10.0,21.0,17.0,11.0,0.525,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
814269,chr2_178113076_+/m54329U_210814_130637/3873255...,-1,117.0,2.0,11.0,7.0,14.0,14.0,8.0,0.350,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1429714,1429714,1429714
814270,chr14_76717786_-/m64076_221119_202646/11809704...,-1,96.0,9.0,7.0,14.0,19.0,19.0,16.0,0.475,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1017681,1017681,1017681
814271,chr16_85169019_+/m54329U_210813_020940/7805359...,-1,81.0,6.0,5.0,2.0,20.0,8.0,4.0,0.500,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1102325,1102325,1102325
814272,chr12_121994066_-/m64076_210328_012155/6429360...,-1,67.0,3.0,2.0,4.0,10.0,7.0,5.0,0.250,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,934069,934069,934069


## archive

### feature info collection

In [ ]:
# working
def agg_features(x, feature_cols):
    '''Collects features. Returns a df with feature info per unique group.'''
    feature_cols = ["msp_size",
                    "left_m6a_count", "right_m6a_count", "motif_m6a_count", # m6a count
                    "left_AT_count", "right_AT_count", "motif_AT_count", # AT count
                    "left_AT_prop", "right_AT_prop", "motif_AT_prop", # proportion of bases that are AT
                    "left_m6a_prop", "right_m6a_prop", "motif_m6a_prop", # proportion of ATs that are methylated
                    "total_m6a_prop", # proportion of ATs that are methylated in subseq_sequence
                   ] + feature_cols
    
    # dict to hold info
    d = dict((col, 0) for col in feature_cols)
    
    # msp size
    d[feature_cols[0]] = x["msp_size"].unique()[0]
    
    # m6a count flanking left, right, motif
    d[feature_cols[1]] = (x["centered_end"] < 0).sum()
    d[feature_cols[2]] = (x["centered_start"] >= 35).sum()
    d[feature_cols[3]] = ((x["centered_start"] >= 0) & (x["centered_end"] <= 35)).sum()
    
    # sequences
    center = 100
    motif_len = 35
    subseq = x["subset_sequence"].unique()[0]
    subseq_motif = subseq[center:(center+motif_len)]
    # length of flank in bp
    flank_len = 40
    # flank left & flank_right
    subseq_l = subseq[center-flank_len:center]
    subseq_r = subseq[center+motif_len:center+motif_len+flank_len]
    
    # AT count
    d[feature_cols[4]] = (subseq_l.count("A") + subseq_l.count("T"))
    d[feature_cols[5]] = (subseq_r.count("A") + subseq_r.count("T"))
    d[feature_cols[6]] = (subseq_motif.count("A") + subseq_motif.count("T"))
    
    # proportion of bases that are AT
    d[feature_cols[7]] = (subseq_l.count("A") + subseq_l.count("T"))/flank_len
    d[feature_cols[8]] = (subseq_r.count("A") + subseq_r.count("T"))/flank_len
    d[feature_cols[9]] = (subseq_motif.count("A") + subseq_motif.count("T"))/motif_len
    
    # proportion of methylated ATs (m6a_count/AT_count)
    d[feature_cols[10]] = d[feature_cols[1]]/d[feature_cols[4]]
    d[feature_cols[11]] = d[feature_cols[2]]/d[feature_cols[5]]
    d[feature_cols[12]] = d[feature_cols[3]]/d[feature_cols[6]]
    d[feature_cols[13]] = (x["centered_position_type"] == "m6a").sum()/(subseq.count("A") + subseq.count("T"))
    
    
    #----- k-mer info -----#
    
    # m6a bool
    m6a_bool = x[(x["centered_start"] >= 0) & (x["centered_end"] <= 35)]["centered_start"].values
    m6a_bool = [1 if (i in m6a_bool) else 0 for i in range(0, 35)]
    
    # decompose motif seq into k-mers
    motif_kmers = get_kmers(subseq_motif, 3)
    m6a_kmers = get_kmers(m6a_bool, 3)
    # make bool of ATs in motif seq
    motifis_AT = [("A" in kmer or "T" in kmer) for kmer in motif_kmers]
    
    # drop motif instances without an AT
    motif_kmers = list(np.array(motif_kmers)[motifis_AT])
    m6a_kmers = list(np.array(m6a_kmers)[motifis_AT])
    
    # make kmer_dict
    kmer_info = defaultdict(lambda: defaultdict(int))

    for kmer, m6as in zip(motif_kmers, m6a_kmers):
        #d[kmer]["count"] += 1
        #d[kmer]["m6a"] += sum(m6as)
        
        kmer_keys = [kmer+"_count", kmer+"_m6a"]
        d[kmer_keys[0]] += 1
        d[kmer_keys[1]] += sum(m6as)
    
    return pd.Series(d, index=list(d.keys()))

In [ ]:
##### FUNCTIONING #####
def agg_features(x):
    '''Collects features. Returns a df with feature info per unique group.'''
    feature_cols = ["msp_size",
                    "left_m6a_count", "right_m6a_count", "motif_m6a_count", # m6a count
                    "left_AT_count", "right_AT_count", "motif_AT_count", # AT count
                    "left_AT_prop", "right_AT_prop", "motif_AT_prop", # proportion of bases that are AT
                    "left_m6a_prop", "right_m6a_prop", "motif_m6a_prop", # proportion of ATs that are methylated
                    "total_m6a_prop", # proportion of ATs that are methylated in subseq_sequence
                   ] 
    d = {}
    # msp size
    d[feature_cols[0]] = x["msp_size"].unique()[0]
    
    # m6a count flanking left, right, motif
    d[feature_cols[1]] = (x["centered_end"] < 0).sum()
    d[feature_cols[2]] = (x["centered_start"] >= 35).sum()
    d[feature_cols[3]] = ((x["centered_start"] >= 0) & (x["centered_end"] <= 35)).sum()
    
    # sequences
    center = 100
    motif_len = 35
    subseq = x["subset_sequence"].unique()[0]
    subseq_motif = subseq[center:(center+motif_len)]
    # length of flank in bp
    flank_len = 40
    # flank left & flank_right
    subseq_l = subseq[center-flank_len:center]
    subseq_r = subseq[center+motif_len:center+motif_len+flank_len]
    
    # AT count
    d[feature_cols[4]] = (subseq_l.count("A") + subseq_l.count("T"))
    d[feature_cols[5]] = (subseq_r.count("A") + subseq_r.count("T"))
    d[feature_cols[6]] = (subseq_motif.count("A") + subseq_motif.count("T"))
    
    # proportion of bases that are AT
    d[feature_cols[7]] = (subseq_l.count("A") + subseq_l.count("T"))/flank_len
    d[feature_cols[8]] = (subseq_r.count("A") + subseq_r.count("T"))/flank_len
    d[feature_cols[9]] = (subseq_motif.count("A") + subseq_motif.count("T"))/motif_len
    
    # proportion of methylated ATs (m6a_count/AT_count)
    d[feature_cols[10]] = d[feature_cols[1]]/d[feature_cols[4]]
    d[feature_cols[11]] = d[feature_cols[2]]/d[feature_cols[5]]
    d[feature_cols[12]] = d[feature_cols[3]]/d[feature_cols[6]]
    d[feature_cols[13]] = (x["centered_position_type"] == "m6a").sum()/(subseq.count("A") + subseq.count("T"))
    
    return pd.Series(d, index=feature_cols)

In [16]:
#----- SAVING OLD VERSION -----#

%%time
# aggregate features
res = df.groupby(grouping_cols).apply(agg_features).reset_index()
# change col dtypes for memory
res[res.columns.tolist()[2:]] = res[res.columns.tolist()[2:]].apply(pd.to_numeric, downcast="float")
print("Total rows: " + "{:,}".format(res.shape[0]))
print("Total columns: " + "{:,}".format(res.shape[1]))

Total rows: 1,986
Total columns: 16
CPU times: user 2.11 s, sys: 2.5 ms, total: 2.11 s
Wall time: 2.12 s


### make k-mer algo

k-mer features (1 column each)
* [ ] 3-mer k-mer count within canonical motif (n/a: exclude k-mers without AT)
* [ ] m6a count within each k-mer
> $$ m6a\ kmer\ prop = m6a\ count / (AT\ count * kmer counts)$$

also need to get the m6a instances and map them to the motif sequence
<br>map sequence positions to m6a coords & count while iterating through sequence
<br>create second matching bool array of m6a at each site

In [18]:
# n k-mers
k = 3

base_map = {"A": 0, "C": 1, "G": 2, "T": 3}

In [19]:
def get_motif_seq(subset_sequence):
    '''Get motif sequence from subset_sequence.'''
    center_idx = 100
    motif_len = 35
    motif_seq = subset_sequence[center:(center+motif_len)]
    return motif_seq

In [20]:
def get_kmers(seq, k):
    '''Decompose the input sequence (str) into k-mers (str).'''
    num_kmers = len(seq) - k + 1
    seqs = [seq[i:i+k] for i in range(num_kmers)]
    return seqs

In [21]:
def seq_to_int_tuple(seq):
    '''Convert the input DNA sequence (str) to a int-encoded tuple.'''
    base_map = {"A": 0, "C": 1, "G": 2, "T": 3}
    int_tuple = tuple(base_map[base] for base in seq)
    return int_tuple

In [28]:
# make alphabetical list of all possible k-mers (kmer_count, kmer_m6a_prop)
kmer_cols = sorted(["".join(p) for p in itertools.permutations("ACGT", k)])
print(len(kmer_cols))
kmer_cols = list(itertools.chain(*[[kmer+"_count", kmer+"_m6a"] for kmer in kmer_cols]))
print(len(kmer_cols))

24
48


In [315]:
from collections import OrderedDict

In [316]:
def make_kmer_info(m6a_pos, subset_sequence, k=3):
    # make bool of m6a in motif_seq
    #m6a_bool = d[(d["centered_start"] >= 0) & (d["centered_end"] <= 35)]["centered_start"].values
    m6a_bool = [1 if (i in m6a_pos) else 0 for i in range(0, 35)]

    # get motif sequence
    motif_seq = get_motif_seq(subset_sequence)
    # make bool of ATs in motif seq
    motifis_AT = np.array([(base == "A" or base == "T") for base in motif_seq], dtype=int)
    
    # decompose motif seq into k-mers
    motif_kmers = get_kmers(motif_seq, 3)
    m6a_kmers = get_kmers(m6a_bool, 3)
    # make bool of ATs in motif seq
    motifis_AT = [("A" in kmer or "T" in kmer) for kmer in motif_kmers]
    
    # drop motif instances without an AT
    motif_kmers = list(np.array(motif_kmers)[motifis_AT])
    m6a_kmers = list(np.array(m6a_kmers)[motifis_AT])
        
    # make k-mer dict (new)
    kmer_info = defaultdict(int)

    for kmer, m6as in zip(motif_kmers, m6a_kmers):
        kmer_keys = [kmer+"_count", kmer+"_m6a"]
        kmer_info[kmer_keys[0]] += 1
        kmer_info[kmer_keys[1]] += sum(m6as)
    
    kmer_info = OrderedDict(sorted(kmer_info.items()))
    
    return kmer_info

In [40]:
# get sample rows
row_idx = 2
d = df[(df["motif_name"] == "chr1_10173651_+") & (df["query_name"] == "m54329U_210323_190418/50594832/ccs")]
print(d.shape)

# get only m6a's within motif
#d_m6a = d[(d["centered_start"] >= 0) & (d["centered_end"] <= 35)]
#d_m6a[["centered_start", "centered_end"]]

(8, 15)


In [41]:
# test sequence
subset_sequence = d["subset_sequence"].unique()[0]

# get motif seq
center = 100
motif_len = 35
motif_seq = subset_sequence[center:(center+motif_len)]
# make bool of ATs in motif seq
motifis_AT = [(base == "A" or base == "T") for base in motif_seq]
print("motif seq: {}".format(len(motif_seq)))

motif seq: 35


In [42]:
# make bool of m6a in motif_seq
m6a_bool = d[(d["centered_start"] >= 0) & (d["centered_end"] <= 35)]["centered_start"].values
m6a_bool = [1 if (i in m6a_bool) else 0 for i in range(0, 35)]

# get motif sequence
motif_seq = get_motif_seq(subset_sequence)
# make bool of ATs in motif seq
motifis_AT = np.array([(base == "A" or base == "T") for base in motif_seq], dtype=int)

In [234]:
# decompose motif seq into k-mers
motif_kmers = get_kmers(motif_seq, 3)
m6a_kmers = get_kmers(m6a_bool, 3)
# make bool of ATs in motif seq
motifis_AT = [("A" in kmer or "T" in kmer) for kmer in motif_kmers]

In [211]:
for x, y in zip(motif_kmers, motifis_AT):
    print("{} | {}".format(x, y))

GTA | True
TAG | True
AGC | True
GCT | True
CTG | True
TGG | True
GGC | False
GCG | False
CGT | True
GTC | True
TCT | True
CTC | True
TCC | True
CCA | True
CAG | True
AGC | True
GCC | False
CCC | False
CCA | True
CAG | True
AGC | True
GCC | False
CCA | True
CAG | True
AGA | True
GAG | True
AGG | True
GGT | True
GTC | True
TCG | True
CGG | False
GGT | True
GTG | True


In [235]:
motif_kmers = list(np.array(motif_kmers)[motifis_AT])
m6a_kmers = list(np.array(m6a_kmers)[motifis_AT])

In [241]:
# make kmer_dict
kmer_info = defaultdict(lambda: defaultdict(int))

for kmer, m6as in zip(motif_kmers, m6a_kmers):
    kmer_info[kmer]["count"] += 1
    kmer_info[kmer]["m6a"] += sum(m6as)

In [242]:
for k, v in kmer_info:

defaultdict(<function __main__.<lambda>()>,
            {'GTA': defaultdict(int, {'count': 1, 'm6a': 0}),
             'TAG': defaultdict(int, {'count': 1, 'm6a': 0}),
             'AGC': defaultdict(int, {'count': 3, 'm6a': 2}),
             'GCT': defaultdict(int, {'count': 1, 'm6a': 0}),
             'CTG': defaultdict(int, {'count': 1, 'm6a': 0}),
             'TGG': defaultdict(int, {'count': 1, 'm6a': 0}),
             'CGT': defaultdict(int, {'count': 1, 'm6a': 0}),
             'GTC': defaultdict(int, {'count': 2, 'm6a': 0}),
             'TCT': defaultdict(int, {'count': 1, 'm6a': 0}),
             'CTC': defaultdict(int, {'count': 1, 'm6a': 0}),
             'TCC': defaultdict(int, {'count': 1, 'm6a': 0}),
             'CCA': defaultdict(int, {'count': 3, 'm6a': 3}),
             'CAG': defaultdict(int, {'count': 3, 'm6a': 3}),
             'AGA': defaultdict(int, {'count': 1, 'm6a': 2}),
             'GAG': defaultdict(int, {'count': 1, 'm6a': 1}),
             'AGG': defaul

In [212]:
print(len(motif_kmers))
print(len(m6a_kmers))
print(len(motifis_AT))
print(len(motif_kmers[motifis_AT]))
print(len(m6a_kmers[motifis_AT]))

33
33
33


TypeError: list indices must be integers or slices, not list

In [44]:
# remove k-mers (& info) without AT's
for i in sorted(np.where(np.array(kmer_ATs) == 0)[0], reverse=True):
    del motif_kmers[i]
    del m6a_kmers[i]
    del kmer_ATs[i]

In [248]:
kmer_info = defaultdict(lambda: defaultdict(int))

for kmer, m6as in zip(motif_kmers, m6a_kmers):
    kmer_info[kmer]["count"] += 1
    kmer_info[kmer]["m6a"] += sum(m6as)

kmer_info

defaultdict(<function __main__.<lambda>()>,
            {'GTA': defaultdict(int, {'count': 1, 'm6a': 0}),
             'TAG': defaultdict(int, {'count': 1, 'm6a': 0}),
             'AGC': defaultdict(int, {'count': 3, 'm6a': 2}),
             'GCT': defaultdict(int, {'count': 1, 'm6a': 0}),
             'CTG': defaultdict(int, {'count': 1, 'm6a': 0}),
             'TGG': defaultdict(int, {'count': 1, 'm6a': 0}),
             'CGT': defaultdict(int, {'count': 1, 'm6a': 0}),
             'GTC': defaultdict(int, {'count': 2, 'm6a': 0}),
             'TCT': defaultdict(int, {'count': 1, 'm6a': 0}),
             'CTC': defaultdict(int, {'count': 1, 'm6a': 0}),
             'TCC': defaultdict(int, {'count': 1, 'm6a': 0}),
             'CCA': defaultdict(int, {'count': 3, 'm6a': 3}),
             'CAG': defaultdict(int, {'count': 3, 'm6a': 3}),
             'AGA': defaultdict(int, {'count': 1, 'm6a': 2}),
             'GAG': defaultdict(int, {'count': 1, 'm6a': 1}),
             'AGG': defaul

In [249]:
for k, v in kmer_info.items():
    # total m6a insteances / (all potential m6a's)
    kmer_info[k]["m6a_prop"] = v["m6a"]/((Counter(k)["T"] + Counter(k)["A"]) * v["count"])
    #print(k+str(list(v.keys())))

kmer_info

defaultdict(<function __main__.<lambda>()>,
            {'GTA': defaultdict(int, {'count': 1, 'm6a': 0, 'm6a_prop': 0.0}),
             'TAG': defaultdict(int, {'count': 1, 'm6a': 0, 'm6a_prop': 0.0}),
             'AGC': defaultdict(int,
                         {'count': 3,
                          'm6a': 2,
                          'm6a_prop': 0.6666666666666666}),
             'GCT': defaultdict(int, {'count': 1, 'm6a': 0, 'm6a_prop': 0.0}),
             'CTG': defaultdict(int, {'count': 1, 'm6a': 0, 'm6a_prop': 0.0}),
             'TGG': defaultdict(int, {'count': 1, 'm6a': 0, 'm6a_prop': 0.0}),
             'CGT': defaultdict(int, {'count': 1, 'm6a': 0, 'm6a_prop': 0.0}),
             'GTC': defaultdict(int, {'count': 2, 'm6a': 0, 'm6a_prop': 0.0}),
             'TCT': defaultdict(int, {'count': 1, 'm6a': 0, 'm6a_prop': 0.0}),
             'CTC': defaultdict(int, {'count': 1, 'm6a': 0, 'm6a_prop': 0.0}),
             'TCC': defaultdict(int, {'count': 1, 'm6a': 0, 'm6a_prop': 0.

In [51]:
for x, y in zip(motif_kmers, m6a_kmers):
    print("{} | {}".format(x, y))

GTA | [0, 0, 0]
TAG | [0, 0, 0]
AGC | [0, 0, 0]
GCT | [0, 0, 0]
CTG | [0, 0, 0]
TGG | [0, 0, 0]
CGT | [0, 0, 0]
GTC | [0, 0, 0]
TCT | [0, 0, 0]
CTC | [0, 0, 0]
TCC | [0, 0, 0]
CCA | [0, 0, 1]
CAG | [0, 1, 0]
AGC | [1, 0, 0]
CCA | [0, 0, 1]
CAG | [0, 1, 0]
AGC | [1, 0, 0]
CCA | [0, 0, 1]
CAG | [0, 1, 0]
AGA | [1, 0, 1]
GAG | [0, 1, 0]
AGG | [1, 0, 0]
GGT | [0, 0, 0]
GTC | [0, 0, 0]
TCG | [0, 0, 0]
GGT | [0, 0, 0]
GTG | [0, 0, 0]


In [49]:
sorted(zip(motif_kmers, m6a_kmers))

[('AGA', [1, 0, 1]),
 ('AGC', [0, 0, 0]),
 ('AGC', [1, 0, 0]),
 ('AGC', [1, 0, 0]),
 ('AGG', [1, 0, 0]),
 ('CAG', [0, 1, 0]),
 ('CAG', [0, 1, 0]),
 ('CAG', [0, 1, 0]),
 ('CCA', [0, 0, 1]),
 ('CCA', [0, 0, 1]),
 ('CCA', [0, 0, 1]),
 ('CGT', [0, 0, 0]),
 ('CTC', [0, 0, 0]),
 ('CTG', [0, 0, 0]),
 ('GAG', [0, 1, 0]),
 ('GCT', [0, 0, 0]),
 ('GGT', [0, 0, 0]),
 ('GGT', [0, 0, 0]),
 ('GTA', [0, 0, 0]),
 ('GTC', [0, 0, 0]),
 ('GTC', [0, 0, 0]),
 ('GTG', [0, 0, 0]),
 ('TAG', [0, 0, 0]),
 ('TCC', [0, 0, 0]),
 ('TCG', [0, 0, 0]),
 ('TCT', [0, 0, 0]),
 ('TGG', [0, 0, 0])]

In [52]:
kmer_info = defaultdict(int)

for kmer, m6as in sorted(zip(motif_kmers, m6a_kmers)):
    kmer_keys = [kmer+"_count", kmer+"_m6a"]
    kmer_info[kmer_keys[0]] += 1
    kmer_info[kmer_keys[1]] += sum(m6as)

kmer_info

defaultdict(int,
            {'AGA_count': 1,
             'AGA_m6a': 2,
             'AGC_count': 3,
             'AGC_m6a': 2,
             'AGG_count': 1,
             'AGG_m6a': 1,
             'CAG_count': 3,
             'CAG_m6a': 3,
             'CCA_count': 3,
             'CCA_m6a': 3,
             'CGT_count': 1,
             'CGT_m6a': 0,
             'CTC_count': 1,
             'CTC_m6a': 0,
             'CTG_count': 1,
             'CTG_m6a': 0,
             'GAG_count': 1,
             'GAG_m6a': 1,
             'GCT_count': 1,
             'GCT_m6a': 0,
             'GGT_count': 2,
             'GGT_m6a': 0,
             'GTA_count': 1,
             'GTA_m6a': 0,
             'GTC_count': 2,
             'GTC_m6a': 0,
             'GTG_count': 1,
             'GTG_m6a': 0,
             'TAG_count': 1,
             'TAG_m6a': 0,
             'TCC_count': 1,
             'TCC_m6a': 0,
             'TCG_count': 1,
             'TCG_m6a': 0,
             'TCT_count': 1,
  

In [245]:
kmer_info

defaultdict(<function __main__.<lambda>()>,
            {'GTA': defaultdict(int, {'count': 1, 'm6a': 0}),
             'TAG': defaultdict(int, {'count': 1, 'm6a': 0}),
             'AGC': defaultdict(int, {'count': 3, 'm6a': 2}),
             'GCT': defaultdict(int, {'count': 1, 'm6a': 0}),
             'CTG': defaultdict(int, {'count': 1, 'm6a': 0}),
             'TGG': defaultdict(int, {'count': 1, 'm6a': 0}),
             'CGT': defaultdict(int, {'count': 1, 'm6a': 0}),
             'GTC': defaultdict(int, {'count': 2, 'm6a': 0}),
             'TCT': defaultdict(int, {'count': 1, 'm6a': 0}),
             'CTC': defaultdict(int, {'count': 1, 'm6a': 0}),
             'TCC': defaultdict(int, {'count': 1, 'm6a': 0}),
             'CCA': defaultdict(int, {'count': 3, 'm6a': 3}),
             'CAG': defaultdict(int, {'count': 3, 'm6a': 3}),
             'AGA': defaultdict(int, {'count': 1, 'm6a': 2}),
             'GAG': defaultdict(int, {'count': 1, 'm6a': 1}),
             'AGG': defaul

In [401]:
list(kmer_info.keys())

['GTA_count',
 'GTA_m6a',
 'TAG_count',
 'TAG_m6a',
 'AGC_count',
 'AGC_m6a',
 'GCT_count',
 'GCT_m6a',
 'CTG_count',
 'CTG_m6a',
 'TGG_count',
 'TGG_m6a',
 'CGT_count',
 'CGT_m6a',
 'GTC_count',
 'GTC_m6a',
 'TCT_count',
 'TCT_m6a',
 'CTC_count',
 'CTC_m6a',
 'TCC_count',
 'TCC_m6a',
 'CCA_count',
 'CCA_m6a',
 'CAG_count',
 'CAG_m6a',
 'AGA_count',
 'AGA_m6a',
 'GAG_count',
 'GAG_m6a',
 'AGG_count',
 'AGG_m6a',
 'GGT_count',
 'GGT_m6a',
 'TCG_count',
 'TCG_m6a',
 'GTG_count',
 'GTG_m6a']

In [295]:
pd.Series(kmer_info, index=kmer_info.keys())

GTA_count    1
GTA_m6a      0
TAG_count    1
TAG_m6a      0
AGC_count    3
AGC_m6a      2
GCT_count    1
GCT_m6a      0
CTG_count    1
CTG_m6a      0
TGG_count    1
TGG_m6a      0
CGT_count    1
CGT_m6a      0
GTC_count    2
GTC_m6a      0
TCT_count    1
TCT_m6a      0
CTC_count    1
CTC_m6a      0
TCC_count    1
TCC_m6a      0
CCA_count    3
CCA_m6a      3
CAG_count    3
CAG_m6a      3
AGA_count    1
AGA_m6a      2
GAG_count    1
GAG_m6a      1
AGG_count    1
AGG_m6a      1
GGT_count    2
GGT_m6a      0
TCG_count    1
TCG_m6a      0
GTG_count    1
GTG_m6a      0
dtype: int64

In [148]:
# get all possible k-mers
all_kmers = []
for seq in df["subset_sequence"]:
    kmers = get_kmers(get_motif_seq(seq), 3)
    all_kmers.extend(kmers)
# only unique k-mers
all_kmers = [*set(all_kmers)]
# remove k-mers without AT and sort
all_kmers = sorted([kmer for kmer in all_kmers if ("T" in kmer) or ("A" in kmer)])
print(len(all_kmers))

# make k-mer columns
kmer_cols = list(itertools.chain(*[[kmer+"_count", kmer+"_m6a"] for kmer in all_kmers]))
print(len(kmer_cols))
kmer_cols.count("ATC_count")

56
112


1

In [267]:
def agg_kmers(x, feature_cols):
    '''Collects features. Returns a df with feature info per unique group.'''
    
    # dict to hold info
    d = dict((col, 0) for col in feature_cols)
    
    # m6a bool
    m6a_bool = x[(x["centered_start"] >= 0) & (x["centered_end"] <= 35)]["centered_start"].values
    m6a_bool = [1 if (i in m6a_bool) else 0 for i in range(0, 35)]

    # sequences
    center = 100
    motif_len = 35
    subseq_motif = x["subset_sequence"].unique()[0][center:(center+motif_len)]
    
    # m6a bool
    m6a_bool = x[(x["centered_start"] >= 0) & (x["centered_end"] <= 35)]["centered_start"].values
    m6a_bool = [1 if (i in m6a_bool) else 0 for i in range(0, 35)]
    
    # decompose motif seq into k-mers
    motif_kmers = get_kmers(subseq_motif, 3)
    m6a_kmers = get_kmers(m6a_bool, 3)
    
    # make bool of ATs in motif seq
    motifis_AT = [("A" in kmer or "T" in kmer) for kmer in motif_kmers]
    
    # drop motif instances without an AT
    motif_kmers = list(np.array(motif_kmers)[motifis_AT])
    m6a_kmers = list(np.array(m6a_kmers)[motifis_AT])
    
    # make kmer_dict
    kmer_info = defaultdict(lambda: defaultdict(int))

    for kmer, m6as in zip(motif_kmers, m6a_kmers):
        kmer_info[kmer]["count"] += 1
        kmer_info[kmer]["m6a"] += sum(m6as)    
    
    # get kmer_m6a proportion
    for kmer, v in kmer_info.items():
        d[kmer+"_count"] = v["count"]
        d[kmer+"_m6a_prop"] = v["m6a"]/((Counter(kmer)["T"] + Counter(kmer)["A"]) * v["count"])
    
    return pd.Series(d, index=list(d.keys()))

    
    # dict to hold kmer_info (new)
    #kmer_info = defaultdict(int)

    #for kmer, m6as in sorted(zip(motif_kmers, m6a_kmers)):
    #    kmer_keys = [kmer+"_count", kmer+"_m6a"]
    #    d[kmer_keys[0]] += 1
    #    d[kmer_keys[1]] += sum(m6as)

    # put k-mer info inter output dict
    #for k in kmer_info.keys():
    #    d[k] = kmer_info[k]

    return pd.Series(d, index=list(d.keys()))

In [268]:
# get all possible k-mers
all_kmers = []
for seq in df["subset_sequence"]:
    kmers = get_kmers(get_motif_seq(seq), 3)
    all_kmers.extend(kmers)
# only unique k-mers
all_kmers = [*set(all_kmers)]
# remove k-mers without AT and sort
all_kmers = sorted([kmer for kmer in all_kmers if ("T" in kmer) or ("A" in kmer)])
print(len(all_kmers))

# make k-mer columns
kmer_cols = list(itertools.chain(*[[kmer+"_count", kmer+"_m6a_prop"] for kmer in all_kmers]))
print(len(kmer_cols))

56
112


In [269]:
%%time
# aggregate features
print(len(list(df.iloc[:10000].groupby(grouping_cols).groups.keys())))
kmer_res = df.iloc[:10000].groupby(grouping_cols).apply(lambda x: agg_kmers(x, kmer_cols)).reset_index()
# change col dtypes for memory
#kmer_res[kmer_res.columns.tolist()[2:]] = kmer_res[kmer_res.columns.tolist()[2:]].apply(pd.to_numeric, downcast="float")
print("Total rows: " + "{:,}".format(kmer_res.shape[0]))
#
print("Total columns: " + "{:,}".format(kmer_res.shape[1]))
kmer_res

455
Total rows: 455
Total columns: 114
CPU times: user 682 ms, sys: 0 ns, total: 682 ms
Wall time: 682 ms


,motif_name,query_name,AAA_count,AAA_m6a_prop,AAC_count,AAC_m6a_prop,AAG_count,AAG_m6a_prop,AAT_count,AAT_m6a_prop,...,TGT_count,TGT_m6a_prop,TTA_count,TTA_m6a_prop,TTC_count,TTC_m6a_prop,TTG_count,TTG_m6a_prop,TTT_count,TTT_m6a_prop
0,chr1_1033080_+,m54329U_210323_190418/144837022/ccs,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,1.00,1.0,1.0,0.0,0.0
1,chr1_1033080_+,m54329U_210323_190418/28969124/ccs,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,0.00,1.0,0.0,0.0,0.0
2,chr1_1033080_+,m54329U_210323_190418/61606199/ccs,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,0.25,1.0,0.0,0.0,0.0
3,chr1_1033080_+,m54329U_210323_190418/65799401/ccs,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,0.25,1.0,0.0,0.0,0.0
4,chr1_1033080_+,m54329U_210323_190418/73794045/ccs,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,0.00,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
450,chr1_4585419_-,m54329U_210814_130637/65536960/ccs,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
451,chr1_4585419_-,m64076_210328_012155/151519350/ccs,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,1.0,1.0,0.0,0.0
452,chr1_4585419_-,m64076_221119_202646/166659899/ccs,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
453,chr1_4585419_-,m64076_221119_202646/22088199/ccs,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0


In [96]:
4*12

48

In [65]:
sorted(get_kmers(get_motif_seq(df[(df["motif_name"] == "chr1_1033080_+") & (df["query_name"] == "m54329U_210323_190418/28969124/ccs")]\
["subset_sequence"].unique()[0]), 3))

['AAC',
 'ACG',
 'ACG',
 'CCC',
 'CCG',
 'CCG',
 'CCT',
 'CGG',
 'CGG',
 'CGG',
 'CGT',
 'CGT',
 'CTC',
 'CTT',
 'GAA',
 'GAC',
 'GCC',
 'GCG',
 'GGA',
 'GGA',
 'GGC',
 'GGC',
 'GGG',
 'GGG',
 'GGG',
 'GTT',
 'GTT',
 'TCC',
 'TCT',
 'TGG',
 'TTC',
 'TTC',
 'TTG']

In [ ]:
def agg_kmers(x):
    '''Collects features. Returns a df with feature info per unique group.'''
    
    m6a_bool = x[(x["centered_start"] >= 0) & (x["centered_end"] <= 35)]["centered_start"].values
    m6a_bool = [1 if (i in m6a_bool) else 0 for i in range(0, 35)]

    # get motif sequence
    center = 100
    motif_len = 35
    motif_seq = x["subset_sequence"].unique()[0][center:(center+motif_len)]
    # make bool of ATs in motif seq
    motifis_AT = np.array([(base == "A" or base == "T") for base in motif_seq], dtype=int)
    
    # decompose motif seq into k-mers
    motif_kmers = get_kmers(motif_seq, 3)
    m6a_kmers = get_kmers(m6a_bool, 3)
    kmer_ATs = [kmer.count("A")+kmer.count("T") for kmer in motif_kmers]
    
    # remove k-mers (& info) without AT's
    for i in sorted(np.where(np.array(kmer_ATs) == 0)[0], reverse=True):
        del motif_kmers[i]
        del m6a_kmers[i]
        del kmer_ATs[i]
    
    # dict to hold kmer_info (new)
    kmer_info = defaultdict(int)
    
    for kmer, m6as in zip(motif_kmers, m6a_kmers):
        kmer_keys = [kmer+"_count", kmer+"_m6a"]
        kmer_info[kmer_keys[0]] += 1
        kmer_info[kmer_keys[1]] += sum(m6as)
        
    # get kmer info (old)
    #kmer_info = defaultdict(lambda: defaultdict(int))
    #for kmer, m6as in zip(motif_kmers, m6a_kmers):
    #    kmer_info[kmer]["count"] += 1
    #    kmer_info[kmer]["m6a"] += sum(m6as)
    #print(kmer_info)
    #return kmer_info
    return pd.Series(kmer_info, index=list(kmer_info.keys()))

In [30]:
d_a = {"a": 1, "b": 2, "c": 3}
d_b = {"a": 0, "b": 0, "c": 0}
for k in d_a.keys():
    d_b[k] = d_a[k]
d_b

{'a': 1, 'b': 2, 'c': 3}

In [422]:
print(len(list(df.iloc[:20].groupby(grouping_cols).groups.keys())))

msp_size           354.000000
left_m6a_count       1.000000
right_m6a_count      0.000000
motif_m6a_count      0.000000
left_AT_count        9.000000
right_AT_count      12.000000
motif_AT_count      10.000000
left_AT_prop         0.225000
right_AT_prop        0.300000
motif_AT_prop        0.285714
left_m6a_prop        0.111111
right_m6a_prop       0.000000
motif_m6a_prop       0.000000
total_m6a_prop       0.017241
dtype: float64

In [446]:
df.iloc[135:160]

,motif_name,chrom,centering_position,strand,subset_sequence,reference_start,reference_end,query_name,centered_query_start,centered_query_end,query_length,centered_position_type,centered_start,centered_end,msp_size
274,chr1_1033080_+,chr1,1033080,+,GACCTACGGGGGCGGGTGTGGGGACGCCGGACTACGCGTCAGGAGT...,1023630,1052449,m54329U_210813_020940/85131496/ccs,-9445,19365,28810,m6a,-12,-11,354
275,chr1_1033080_+,chr1,1033080,+,GACCTACGGGGGCGGGTGTGGGGACGCCGGACTACGCGTCAGGAGT...,1023630,1052449,m54329U_210813_020940/85131496/ccs,-9445,19365,28810,m6a,-10,-9,354
276,chr1_1033080_+,chr1,1033080,+,GACCTACGGGGGCGGGTGTGGGGACGCCGGACTACGCGTCAGGAGT...,1023630,1052449,m54329U_210813_020940/85131496/ccs,-9445,19365,28810,m6a,-7,-6,354
277,chr1_1033080_+,chr1,1033080,+,GACCTACGGGGGCGGGTGTGGGGACGCCGGACTACGCGTCAGGAGT...,1023630,1052449,m54329U_210813_020940/85131496/ccs,-9445,19365,28810,m6a,-5,-4,354
278,chr1_1033080_+,chr1,1033080,+,GACCTACGGGGGCGGGTGTGGGGACGCCGGACTACGCGTCAGGAGT...,1023630,1052449,m54329U_210813_020940/85131496/ccs,-9445,19365,28810,m6a,4,5,354
279,chr1_1033080_+,chr1,1033080,+,GACCTACGGGGGCGGGTGTGGGGACGCCGGACTACGCGTCAGGAGT...,1023630,1052449,m54329U_210813_020940/85131496/ccs,-9445,19365,28810,m6a,7,8,354
280,chr1_1033080_+,chr1,1033080,+,GACCTACGGGGGCGGGTGTGGGGACGCCGGACTACGCGTCAGGAGT...,1023630,1052449,m54329U_210813_020940/85131496/ccs,-9445,19365,28810,m6a,8,9,354
281,chr1_1033080_+,chr1,1033080,+,GACCTACGGGGGCGGGTGTGGGGACGCCGGACTACGCGTCAGGAGT...,1023630,1052449,m54329U_210813_020940/85131496/ccs,-9445,19365,28810,m6a,20,21,354
282,chr1_1033080_+,chr1,1033080,+,GACCTACGGGGGCGGGTGTGGGGACGCCGGACTACGCGTCAGGAGT...,1023630,1052449,m54329U_210813_020940/85131496/ccs,-9445,19365,28810,m6a,22,23,354
283,chr1_1033080_+,chr1,1033080,+,GACCTACGGGGGCGGGTGTGGGGACGCCGGACTACGCGTCAGGAGT...,1023630,1052449,m54329U_210813_020940/85131496/ccs,-9445,19365,28810,m6a,23,24,354


In [470]:
%%time
# aggregate features
print(len(list(df.iloc[130:160].groupby(grouping_cols).groups.keys())))
kmer_res = df.iloc[130:160].groupby(grouping_cols).apply(agg_kmers).reset_index()
# change col dtypes for memory
#kmer_res[kmer_res.columns.tolist()[2:]] = kmer_res[kmer_res.columns.tolist()[2:]].apply(pd.to_numeric, downcast="float")
#print("Total rows: " + "{:,}".format(kmer_res.shape[0]))
#print("Total columns: " + "{:,}".format(kmer_res.shape[1]))
kmer_res

3
CPU times: user 16.4 ms, sys: 5.31 ms, total: 21.7 ms
Wall time: 18.1 ms


,motif_name,query_name,level_2,0
0,chr1_1033080_+,m54329U_210813_020940/85131496/ccs,GGA_count,2
1,chr1_1033080_+,m54329U_210813_020940/85131496/ccs,GGA_m6a,1
2,chr1_1033080_+,m54329U_210813_020940/85131496/ccs,GAC_count,1
3,chr1_1033080_+,m54329U_210813_020940/85131496/ccs,GAC_m6a,1
4,chr1_1033080_+,m54329U_210813_020940/85131496/ccs,ACG_count,2
...,...,...,...,...
85,chr1_1033080_+,m64076_210328_012155/84345303/ccs,CTT_m6a,0
86,chr1_1033080_+,m64076_210328_012155/84345303/ccs,TTG_count,1
87,chr1_1033080_+,m64076_210328_012155/84345303/ccs,TTG_m6a,0
88,chr1_1033080_+,m64076_210328_012155/84345303/ccs,TGG_count,1


In [333]:
# aggregate features
kmer_res = df.iloc[:100].groupby(grouping_cols).apply(agg_features).reset_index()

# change col dtypes for memory
#res_small[res_small.columns.tolist()[2:]] = res_small[res_small.columns.tolist()[2:]].apply(pd.to_numeric, downcast="float")
print("Total rows: " + "{:,}".format(kmer_res.shape[0]))
print("Total columns: " + "{:,}".format(kmer_res.shape[1]))

kmer_res.head()

Total rows: 9
Total columns: 32
CPU times: user 21.5 ms, sys: 5.54 ms, total: 27 ms
Wall time: 23.3 ms


,motif_name,query_name,GGA_count,GGA_m6a,GAC_count,GAC_m6a,ACG_count,ACG_m6a,CGT_count,CGT_m6a,...,CTC_count,CTC_m6a,TCT_count,TCT_m6a,CTT_count,CTT_m6a,TTG_count,TTG_m6a,TGG_count,TGG_m6a
0,chr1_1033080_+,m54329U_210323_190418/28969124/ccs,2,0,1,0,2,0,2,0,...,1,0,1,0,1,0,1,0,1,0
1,chr1_1033080_+,m54329U_210323_190418/65799401/ccs,2,0,1,0,2,0,2,1,...,1,0,1,0,1,0,1,0,1,0
2,chr1_1033080_+,m54329U_210323_190418/92997266/ccs,2,0,1,0,2,0,2,0,...,1,0,1,0,1,0,1,0,1,0
3,chr1_1033080_+,m54329U_210810_004956/66257203/ccs,2,0,1,0,2,0,2,0,...,1,0,1,0,1,0,1,0,1,0
4,chr1_1033080_+,m54329U_210813_020940/173737659/ccs,2,0,1,0,2,0,2,1,...,1,0,1,0,1,0,1,0,1,0


In [318]:
%%time
# aggregate features
res_small = df.iloc[:30].groupby(grouping_cols).apply(agg_features).reset_index()
# change col dtypes for memory
#res_small[res_small.columns.tolist()[2:]] = res_small[res_small.columns.tolist()[2:]].apply(pd.to_numeric, downcast="float")
print("Total rows: " + "{:,}".format(res_small.shape[0]))
print("Total columns: " + "{:,}".format(res_small.shape[1]))

res_small.head()

Total rows: 3
Total columns: 32
CPU times: user 12.8 ms, sys: 2.16 ms, total: 14.9 ms
Wall time: 12.6 ms


,motif_name,query_name,GGA_count,GGA_m6a,GAC_count,GAC_m6a,ACG_count,ACG_m6a,CGT_count,CGT_m6a,...,CTC_count,CTC_m6a,TCT_count,TCT_m6a,CTT_count,CTT_m6a,TTG_count,TTG_m6a,TGG_count,TGG_m6a
0,chr1_1033080_+,m54329U_210323_190418/65799401/ccs,2,0,1,0,2,0,2,1,...,1,0,1,0,1,0,1,0,1,0
1,chr1_1033080_+,m54329U_210813_020940/173737659/ccs,2,0,1,0,2,0,2,1,...,1,0,1,0,1,0,1,0,1,0
2,chr1_1033080_+,m54329U_210813_020940/24183559/ccs,2,0,1,0,2,0,2,0,...,1,0,1,0,1,0,1,0,1,0


In [156]:
print("motif kmers: {}\nm6a kmers: {}\nkmer_ATs: {}".format(len(motif_kmers), len(m6a_kmers), len(kmer_ATs)))

motif kmers: 33
m6a kmers: 33
kmer_ATs: 33


In [158]:
print("motif kmers: {}\nm6a kmers: {}\nkmer_ATs: {}".format(len(motif_kmers), len(m6a_kmers), len(kmer_ATs)))

motif kmers: 27
m6a kmers: 27
kmer_ATs: 27


In [167]:
collections.Counter(motif_kmers)

Counter({'GTA': 1,
         'TAG': 1,
         'AGC': 3,
         'GCT': 1,
         'CTG': 1,
         'TGG': 1,
         'CGT': 1,
         'GTC': 2,
         'TCT': 1,
         'CTC': 1,
         'TCC': 1,
         'CCA': 3,
         'CAG': 3,
         'AGA': 1,
         'GAG': 1,
         'AGG': 1,
         'GGT': 2,
         'TCG': 1,
         'GTG': 1})

In [358]:
test_keys = list(df.iloc[:10].groupby(grouping_cols).groups.keys())

test_idx = 0
d_test = df[(df["motif_name"] == test_keys[test_idx][0]) & (df["query_name"] == test_keys[test_idx][1])]

# using only m6a's within motif
kmer_info_test = make_kmer_info(
    d_test[(d_test["centered_start"] >= 0) & (d_test["centered_end"] <= 35)]["centered_start"].values, 
    d_test["subset_sequence"].unique()[0])

kmer_info_test

defaultdict(<function __main__.make_kmer_info.<locals>.<lambda>()>,
            {'GGA': defaultdict(int, {'count': 2, 'm6a': 0}),
             'GAC': defaultdict(int, {'count': 1, 'm6a': 0}),
             'ACG': defaultdict(int, {'count': 2, 'm6a': 0}),
             'CGT': defaultdict(int, {'count': 2, 'm6a': 1}),
             'GTT': defaultdict(int, {'count': 2, 'm6a': 1}),
             'TTC': defaultdict(int, {'count': 2, 'm6a': 1}),
             'TCC': defaultdict(int, {'count': 1, 'm6a': 0}),
             'GAA': defaultdict(int, {'count': 1, 'm6a': 0}),
             'AAC': defaultdict(int, {'count': 1, 'm6a': 0}),
             'CCT': defaultdict(int, {'count': 1, 'm6a': 0}),
             'CTC': defaultdict(int, {'count': 1, 'm6a': 0}),
             'TCT': defaultdict(int, {'count': 1, 'm6a': 0}),
             'CTT': defaultdict(int, {'count': 1, 'm6a': 0}),
             'TTG': defaultdict(int, {'count': 1, 'm6a': 0}),
             'TGG': defaultdict(int, {'count': 1, 'm6a': 0})})

In [489]:
kmer_info = dict.fromkeys(kmer_cols, 0)

for kmer, m6as in zip(motif_kmers, m6a_kmers):
    kmer_info[kmer+"_count"] = kmer_info.get(kmer+"_count", 1) + 1
    kmer_info[kmer+"_m6a"] = kmer_info.get(kmer+"_m6a", 1) + 1

In [491]:
kmer_info

{'ACG_count': 0,
 'ACG_m6a': 0,
 'ACT_count': 0,
 'ACT_m6a': 0,
 'AGC_count': 3,
 'AGC_m6a': 3,
 'AGT_count': 0,
 'AGT_m6a': 0,
 'ATC_count': 0,
 'ATC_m6a': 0,
 'ATG_count': 0,
 'ATG_m6a': 0,
 'CAG_count': 3,
 'CAG_m6a': 3,
 'CAT_count': 0,
 'CAT_m6a': 0,
 'CGA_count': 0,
 'CGA_m6a': 0,
 'CGT_count': 1,
 'CGT_m6a': 1,
 'CTA_count': 0,
 'CTA_m6a': 0,
 'CTG_count': 1,
 'CTG_m6a': 1,
 'GAC_count': 0,
 'GAC_m6a': 0,
 'GAT_count': 0,
 'GAT_m6a': 0,
 'GCA_count': 0,
 'GCA_m6a': 0,
 'GCT_count': 1,
 'GCT_m6a': 1,
 'GTA_count': 1,
 'GTA_m6a': 1,
 'GTC_count': 2,
 'GTC_m6a': 2,
 'TAC_count': 0,
 'TAC_m6a': 0,
 'TAG_count': 1,
 'TAG_m6a': 1,
 'TCA_count': 0,
 'TCA_m6a': 0,
 'TCG_count': 1,
 'TCG_m6a': 1,
 'TGA_count': 0,
 'TGA_m6a': 0,
 'TGC_count': 0,
 'TGC_m6a': 0,
 'TGG_count': 2,
 'TGG_m6a': 2,
 'TCT_count': 2,
 'TCT_m6a': 2,
 'CTC_count': 2,
 'CTC_m6a': 2,
 'TCC_count': 2,
 'TCC_m6a': 2,
 'CCA_count': 4,
 'CCA_m6a': 4,
 'AGA_count': 2,
 'AGA_m6a': 2,
 'GAG_count': 2,
 'GAG_m6a': 2,
 'AGG_co

In [490]:
len(kmer_info)

68

In [212]:
kmer_info = collections.defaultdict(lambda: collections.defaultdict(int))

for kmer, m6as in zip(motif_kmers, m6a_kmers):
    kmer_info[kmer] += [1, sum(m6as)]
    kmer_info[kmer] += sum(m6as)
    
kmer_info

TypeError: unsupported operand type(s) for +=: 'collections.defaultdict' and 'list'

In [209]:
kmer_info

defaultdict(<function __main__.<lambda>()>,
            {'GTA': defaultdict(int, {'kmer_count': 1, 'm6a_count': 0}),
             'TAG': defaultdict(int, {'kmer_count': 1, 'm6a_count': 0}),
             'AGC': defaultdict(int, {'kmer_count': 3, 'm6a_count': 2}),
             'GCT': defaultdict(int, {'kmer_count': 1, 'm6a_count': 0}),
             'CTG': defaultdict(int, {'kmer_count': 1, 'm6a_count': 0}),
             'TGG': defaultdict(int, {'kmer_count': 1, 'm6a_count': 0}),
             'CGT': defaultdict(int, {'kmer_count': 1, 'm6a_count': 0}),
             'GTC': defaultdict(int, {'kmer_count': 2, 'm6a_count': 0}),
             'TCT': defaultdict(int, {'kmer_count': 1, 'm6a_count': 0}),
             'CTC': defaultdict(int, {'kmer_count': 1, 'm6a_count': 0}),
             'TCC': defaultdict(int, {'kmer_count': 1, 'm6a_count': 0}),
             'CCA': defaultdict(int, {'kmer_count': 3, 'm6a_count': 3}),
             'CAG': defaultdict(int, {'kmer_count': 3, 'm6a_count': 3}),
       

In [196]:
collections.defaultdict(lambda: collections.defaultdict(int))

defaultdict(<function __main__.<lambda>()>, {})

In [182]:
kmer_info

defaultdict(list,
            {'GTA': [1, 0],
             'TAG': [1, 0],
             'AGC': [1, 0, 1, 1, 1, 1],
             'GCT': [1, 0],
             'CTG': [1, 0],
             'TGG': [1, 0],
             'CGT': [1, 0],
             'GTC': [1, 0, 1, 0],
             'TCT': [1, 0],
             'CTC': [1, 0],
             'TCC': [1, 0],
             'CCA': [1, 1, 1, 1, 1, 1],
             'CAG': [1, 1, 1, 1, 1, 1],
             'AGA': [1, 2],
             'GAG': [1, 1],
             'AGG': [1, 1],
             'GGT': [1, 0, 1, 0],
             'TCG': [1, 0],
             'GTG': [1, 0]})

In [138]:
for x, y in zip(motif_kmers, kmer_ATs):
    print(x, y)

GTA 2
TAG 2
AGC 1
GCT 1
CTG 1
TGG 1
GGC 0
GCG 0
CGT 1
GTC 1
TCT 2
CTC 1
TCC 1
CCA 1
CAG 1
AGC 1
GCC 0
CCC 0
CCA 1
CAG 1
AGC 1
GCC 0
CCA 1
CAG 1
AGA 2
GAG 1
AGG 1
GGT 1
GTC 1
TCG 1
CGG 0
GGT 1
GTG 1


In [106]:
# decompose motif seq into k-mers
motif_kmers = get_kmers(motif_seq, 3)

In [107]:
motif_kmers

['GTA',
 'TAG',
 'AGC',
 'GCT',
 'CTG',
 'TGG',
 'GGC',
 'GCG',
 'CGT',
 'GTC',
 'TCT',
 'CTC',
 'TCC',
 'CCA',
 'CAG',
 'AGC',
 'GCC',
 'CCC',
 'CCA',
 'CAG',
 'AGC',
 'GCC',
 'CCA',
 'CAG',
 'AGA',
 'GAG',
 'AGG',
 'GGT',
 'GTC',
 'TCG',
 'CGG',
 'GGT',
 'GTG']

In [105]:
# get k-mer counts
pd.Series(motif_kmers).value_counts()

CCA    3
AGC    3
CAG    3
GGT    2
GTC    2
GCC    2
GTA    1
CGG    1
TCG    1
AGG    1
GAG    1
AGA    1
CCC    1
TCC    1
TAG    1
CTC    1
TCT    1
CGT    1
GCG    1
GGC    1
TGG    1
CTG    1
GCT    1
GTG    1
dtype: int64

In [25]:
# decompose motif seq into k-mers
kmer_tuples = get_kmers(motif, 3)
kmer_tuples[:3]

['GTA', 'TAG', 'AGC']

In [26]:
# convert DNA seq's into an int-encoded tuple
kmer_tuples = [seq_to_int_tuple(kmer) for kmer in kmer_tuples]
kmer_tuples[:3]

[(2, 3, 0), (3, 0, 2), (0, 2, 1)]

In [38]:
# make m6a instance array
m6a_pos = d_m6a["centered_start"].values
print(m6a_pos)

# make bool of m6a's in motif seq
m6a_pos = [1 if (i in m6a_pos) else 0 for i in range(0, 35)]
print(m6a_pos)
print(sum(m6a_pos))

[15 20 24 26]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
4


In [86]:
# reindex
d = d.reset_index(drop=True)
d

,motif_name,chrom,centering_position,strand,subset_sequence,reference_start,reference_end,query_name,centered_query_start,centered_query_end,query_length,centered_position_type,centered_start,centered_end,msp_size
0,chr1_10173651_+,chr1,10173651,+,TAATACTGCACTGATTTAGAAACAGAAAGCAAGTCATAGCCTGCAG...,10172558,10185969,m54329U_210323_190418/50594832/ccs,-1093,12330,13423,m6a,-10,-9,51
1,chr1_10173651_+,chr1,10173651,+,TAATACTGCACTGATTTAGAAACAGAAAGCAAGTCATAGCCTGCAG...,10172558,10185969,m54329U_210323_190418/50594832/ccs,-1093,12330,13423,m6a,-8,-7,51
2,chr1_10173651_+,chr1,10173651,+,TAATACTGCACTGATTTAGAAACAGAAAGCAAGTCATAGCCTGCAG...,10172558,10185969,m54329U_210323_190418/50594832/ccs,-1093,12330,13423,m6a,-3,-2,51
3,chr1_10173651_+,chr1,10173651,+,TAATACTGCACTGATTTAGAAACAGAAAGCAAGTCATAGCCTGCAG...,10172558,10185969,m54329U_210323_190418/50594832/ccs,-1093,12330,13423,m6a,15,16,51
4,chr1_10173651_+,chr1,10173651,+,TAATACTGCACTGATTTAGAAACAGAAAGCAAGTCATAGCCTGCAG...,10172558,10185969,m54329U_210323_190418/50594832/ccs,-1093,12330,13423,m6a,20,21,51
5,chr1_10173651_+,chr1,10173651,+,TAATACTGCACTGATTTAGAAACAGAAAGCAAGTCATAGCCTGCAG...,10172558,10185969,m54329U_210323_190418/50594832/ccs,-1093,12330,13423,m6a,24,25,51
6,chr1_10173651_+,chr1,10173651,+,TAATACTGCACTGATTTAGAAACAGAAAGCAAGTCATAGCCTGCAG...,10172558,10185969,m54329U_210323_190418/50594832/ccs,-1093,12330,13423,m6a,26,27,51
7,chr1_10173651_+,chr1,10173651,+,TAATACTGCACTGATTTAGAAACAGAAAGCAAGTCATAGCCTGCAG...,10172558,10185969,m54329U_210323_190418/50594832/ccs,-1093,12330,13423,m6a,40,41,51


In [87]:
# de-compose motif_seq's into their k-mers
kmer_col = d["subset_sequence"].apply(lambda x: get_motif_seq(x))
kmer_col = kmer_col.apply(lambda x: get_kmers(x, 3))
kmer_col

0    [GTA, TAG, AGC, GCT, CTG, TGG, GGC, GCG, CGT, ...
1    [GTA, TAG, AGC, GCT, CTG, TGG, GGC, GCG, CGT, ...
2    [GTA, TAG, AGC, GCT, CTG, TGG, GGC, GCG, CGT, ...
3    [GTA, TAG, AGC, GCT, CTG, TGG, GGC, GCG, CGT, ...
4    [GTA, TAG, AGC, GCT, CTG, TGG, GGC, GCG, CGT, ...
5    [GTA, TAG, AGC, GCT, CTG, TGG, GGC, GCG, CGT, ...
6    [GTA, TAG, AGC, GCT, CTG, TGG, GGC, GCG, CGT, ...
7    [GTA, TAG, AGC, GCT, CTG, TGG, GGC, GCG, CGT, ...
Name: subset_sequence, dtype: object

In [88]:
# encode k-mer seqs as int tuples
kmer_tuples = kmer_col.apply(lambda kmers: [seq_to_int_tuple(kmer) for kmer in kmers])

In [89]:
# create a k-dimensional matrix to store kmer counts
kmer_counts = np.zeros(tuple(4 for i in range(3)), dtype=int)

In [90]:
kmer_counts

array([[[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]],

       [[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]],

       [[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]],

       [[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]]])

In [97]:
kmer_counts[(2, 3, 0)] += 1

In [98]:
kmer_counts

array([[[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]],

       [[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]],

       [[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [1, 0, 0, 0]],

       [[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]]])

In [94]:
# flattens cols to list of tuples
for kmer_tuple in kmer_tuples.explode():
    print(kmer_tuple)

(2, 3, 0)
(3, 0, 2)
(0, 2, 1)
(2, 1, 3)
(1, 3, 2)
(3, 2, 2)
(2, 2, 1)
(2, 1, 2)
(1, 2, 3)
(2, 3, 1)
(3, 1, 3)
(1, 3, 1)
(3, 1, 1)
(1, 1, 0)
(1, 0, 2)
(0, 2, 1)
(2, 1, 1)
(1, 1, 1)
(1, 1, 0)
(1, 0, 2)
(0, 2, 1)
(2, 1, 1)
(1, 1, 0)
(1, 0, 2)
(0, 2, 0)
(2, 0, 2)
(0, 2, 2)
(2, 2, 3)
(2, 3, 1)
(3, 1, 2)
(1, 2, 2)
(2, 2, 3)
(2, 3, 2)
(2, 3, 0)
(3, 0, 2)
(0, 2, 1)
(2, 1, 3)
(1, 3, 2)
(3, 2, 2)
(2, 2, 1)
(2, 1, 2)
(1, 2, 3)
(2, 3, 1)
(3, 1, 3)
(1, 3, 1)
(3, 1, 1)
(1, 1, 0)
(1, 0, 2)
(0, 2, 1)
(2, 1, 1)
(1, 1, 1)
(1, 1, 0)
(1, 0, 2)
(0, 2, 1)
(2, 1, 1)
(1, 1, 0)
(1, 0, 2)
(0, 2, 0)
(2, 0, 2)
(0, 2, 2)
(2, 2, 3)
(2, 3, 1)
(3, 1, 2)
(1, 2, 2)
(2, 2, 3)
(2, 3, 2)
(2, 3, 0)
(3, 0, 2)
(0, 2, 1)
(2, 1, 3)
(1, 3, 2)
(3, 2, 2)
(2, 2, 1)
(2, 1, 2)
(1, 2, 3)
(2, 3, 1)
(3, 1, 3)
(1, 3, 1)
(3, 1, 1)
(1, 1, 0)
(1, 0, 2)
(0, 2, 1)
(2, 1, 1)
(1, 1, 1)
(1, 1, 0)
(1, 0, 2)
(0, 2, 1)
(2, 1, 1)
(1, 1, 0)
(1, 0, 2)
(0, 2, 0)
(2, 0, 2)
(0, 2, 2)
(2, 2, 3)
(2, 3, 1)
(3, 1, 2)
(1, 2, 2)
(2, 2, 3)
(2, 3, 2)
(2, 3, 0)


In [66]:
# flattens cols to list of tuples
for kmer_tuple in kmer_tuples.explode():
    kmer_counts[kmer_tuple] += 1

In [73]:
# lookup k-mer counts for each sequence
count_vectors = kmer_tuples.apply(lambda kmers: [kmer_counts[kmer] for kmer in kmers])

In [77]:
# sum the vector of counts for each sequence
count_sums = count_vectors.apply(np.sum)

In [ ]:
# make k-dimensional matrix to store k-mer counts
kmer_counts = np.zeros(tuple(4 for i in range(3)), dtype=int)

# record each k-mer in the count matrix

In [39]:
m6a_tuples = [m6a_pos[i:i+3] for i in range(0, 35)]
print(m6a_tuples)

[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 1], [0, 1, 0], [1, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 1], [0, 1, 0], [1, 0, 0], [0, 0, 0], [0, 0, 1], [0, 1, 0], [1, 0, 1], [0, 1, 0], [1, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0], [0]]


In [89]:
def get_kmer_sums(df, k, seq_col="subseq_motif", probe_tuples=[]):
    '''Get k-mer counts within a motif-fiber instance.'''
    # de-compose sequences into their k-mers
    kmer_col = df[seq_col].apply(lambda x: get_kmers(x, k))
    
    # encode k-mer seqs as int tuples (eg. CGG to (1, 2, 2))
    kmer_tuples = kmer_col.apply(lambda kmers: [seq_to_int_tuple(kmer) for kmer in kmers])
    
    # create a k-dimentional matrix to store k-mer counts
    kmer_counts = np.zeros(tuple(4 for i in range(k)), dtype=int)
    
    # record each k-mer in the count matrix
    for kmer_tuple in kmer_tuples.explode():
        if kmer_tuple in probe_tuples:
            kmer_counts[kmer_tuple] += 1
    
    # loop k-mer counts for each sequence
    count_vectors = kmer_tuples.apply(lambda kmers: [kmer_counts[kmer] for kmer in kmers])
    
    # sum the vector of counts for each sequence
    count_sums = count_vectors.apply(np.sum)
    
    # success
    return count_sums

In [146]:
# decompose the input array into k-mers
m6a_kmers = [m6a_bool[i:i+3] for i in range(33)]
print(m6a_kmers[:5])

[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]]


In [102]:
# decompose the motif sequence into k-mers
kmer_tuples = get_kmers(motif, 3)
# convert the motif sequence into 
kmer_tuples = [seq_to_int_tuple(kmer) for kmer in kmer_tuples]

In [94]:
# get list of m6a instance positions
m6a_positions = d_m6a["centered_start"].values
print(m6a_positions)

[15 20 24 26]


In [96]:
m6a_bool = [0 if i not in m6a_positions else 1 for i in range(0, 35)]
print(len(m6a_bool))
print(m6a_bool)

35
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]


In [98]:
for x, y in zip(m6a_kmers, get_kmers(motif, 3)):
    print(x, y)

0 GTA
0 TAG
0 AGC
0 GCT
0 CTG
0 TGG
0 GGC
0 GCG
0 CGT
0 GTC
0 TCT
0 CTC
0 TCC
1 CCA
1 CAG
1 AGC
0 GCC
0 CCC
1 CCA
1 CAG
1 AGC
0 GCC
1 CCA
1 CAG
2 AGA
1 GAG
1 AGG
0 GGT
0 GTC
0 TCG
0 CGG
0 GGT
0 GTG


In [99]:
for x, y, z in zip(m6a_kmers, get_kmers(motif, 3), kmer_tuples):
    print("{} | {} | {}".format(x, y, z))

0 | GTA | (2, 3, 0)
0 | TAG | (3, 0, 2)
0 | AGC | (0, 2, 1)
0 | GCT | (2, 1, 3)
0 | CTG | (1, 3, 2)
0 | TGG | (3, 2, 2)
0 | GGC | (2, 2, 1)
0 | GCG | (2, 1, 2)
0 | CGT | (1, 2, 3)
0 | GTC | (2, 3, 1)
0 | TCT | (3, 1, 3)
0 | CTC | (1, 3, 1)
0 | TCC | (3, 1, 1)
1 | CCA | (1, 1, 0)
1 | CAG | (1, 0, 2)
1 | AGC | (0, 2, 1)
0 | GCC | (2, 1, 1)
0 | CCC | (1, 1, 1)
1 | CCA | (1, 1, 0)
1 | CAG | (1, 0, 2)
1 | AGC | (0, 2, 1)
0 | GCC | (2, 1, 1)
1 | CCA | (1, 1, 0)
1 | CAG | (1, 0, 2)
2 | AGA | (0, 2, 0)
1 | GAG | (2, 0, 2)
1 | AGG | (0, 2, 2)
0 | GGT | (2, 2, 3)
0 | GTC | (2, 3, 1)
0 | TCG | (3, 1, 2)
0 | CGG | (1, 2, 2)
0 | GGT | (2, 2, 3)
0 | GTG | (2, 3, 2)


In [117]:
# create a k-dimensional matrix to store k-mer counts
kmer_counts = np.zeros(tuple(4 for i in range(3)), dtype=int)

# record each k-mer in the count matrix
for kmer_tuple in kmer_tuples:
    kmer_counts[kmer_tuple] += 1

4


In [125]:
pd.Series(get_kmers(motif, 3)).value_counts()

CCA    3
AGC    3
CAG    3
GGT    2
GTC    2
GCC    2
GTA    1
CGG    1
TCG    1
AGG    1
GAG    1
AGA    1
CCC    1
TCC    1
TAG    1
CTC    1
TCT    1
CGT    1
GCG    1
GGC    1
TGG    1
CTG    1
GCT    1
GTG    1
dtype: int64

In [137]:
[kmer + "_n" for kmer in kmer_cols]

['ACG_n',
 'ACT_n',
 'AGC_n',
 'AGT_n',
 'ATC_n',
 'ATG_n',
 'CAG_n',
 'CAT_n',
 'CGA_n',
 'CGT_n',
 'CTA_n',
 'CTG_n',
 'GAC_n',
 'GAT_n',
 'GCA_n',
 'GCT_n',
 'GTA_n',
 'GTC_n',
 'TAC_n',
 'TAG_n',
 'TCA_n',
 'TCG_n',
 'TGA_n',
 'TGC_n']

In [126]:
for x, y in zip(get_kmers(motif, 3), [kmer_counts[kmer] for kmer in kmer_tuples]):
    print("{} | {}".format(x, y))

GTA | 1
TAG | 1
AGC | 3
GCT | 1
CTG | 1
TGG | 1
GGC | 1
GCG | 1
CGT | 1
GTC | 2
TCT | 1
CTC | 1
TCC | 1
CCA | 3
CAG | 3
AGC | 3
GCC | 2
CCC | 1
CCA | 3
CAG | 3
AGC | 3
GCC | 2
CCA | 3
CAG | 3
AGA | 1
GAG | 1
AGG | 1
GGT | 2
GTC | 2
TCG | 1
CGG | 1
GGT | 2
GTG | 1


In [122]:
[kmer_counts[kmer] for kmer in kmer_tuples]

[1,
 1,
 3,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 3,
 3,
 3,
 2,
 1,
 3,
 3,
 3,
 2,
 3,
 3,
 1,
 1,
 1,
 2,
 2,
 1,
 1,
 2,
 1]

In [113]:
kmer_counts

array([[[0, 0, 0, 0],
        [0, 0, 0, 0],
        [1, 3, 1, 0],
        [0, 0, 0, 0]],

       [[0, 0, 3, 0],
        [3, 1, 0, 0],
        [0, 0, 1, 1],
        [0, 1, 1, 0]],

       [[0, 0, 1, 0],
        [0, 2, 1, 1],
        [0, 1, 0, 2],
        [1, 2, 1, 0]],

       [[0, 0, 1, 0],
        [0, 1, 1, 1],
        [0, 0, 1, 0],
        [0, 0, 0, 0]]])

In [110]:
# get kmer_sums
print("Getting k-mer sums.")
get_kmer_sums(d_m6a, 3, seq_col="subset_sequence")

Getting k-mer sums.


NameError: name 'probe_tuples' is not defined

### other code

In [ ]:
# sample
negative_queries = ['m54329U_210323_190418/58001273/ccs', 'm54329U_210323_190418/107545216/ccs', 
                    'm54329U_210323_190418/148702267/ccs', 'm54329U_210326_192251/12978426/ccs', 
                    'm54329U_210326_192251/171182291/ccs', 'm54329U_210810_004956/65863987/ccs', 
                    'm54329U_210810_004956/68815147/ccs', 'm54329U_210810_004956/78316132/ccs', 
                    'm54329U_210810_004956/85591510/ccs', 'm54329U_210810_004956/91423766/ccs']

In [ ]:
# combine masks (all msp's containing a motif & m6a's within flanking range)
df_mask = np.logical_or(msp_conds, m6a_range_conds)
print("observations meeting conditions: {}".format(df_mask.sum()))

In [ ]:
# check if motif_name & query_name in msp_groups
df[df[["motif_name", "query_name"]].apply(tuple, 1).isin(msp_groups)]

In [ ]:
# apply mask to df
df = df[df_mask]

In [ ]:
# group by motif & query name
grouping_cols = ["motif_name", "query_name"]
df_grouped = df.groupby(grouping_cols)

# get group names (keys)
group_names = list(df_grouped.groups.keys())
print("different motif-sequence groups: {}".format(len(group_names)))

In [ ]:
df_filt = df[df_mask].groupby(grouping_cols).filter(lambda x: any(x["centered_position_type"] == "msp"))
print(df_filt.shape)

In [ ]:
group_names_1 = group_names
print(len(group_names_1))

In [ ]:
# group by motif & query name
grouping_cols = ["motif_name", "query_name"]
df_grouped = df.groupby(grouping_cols)

# get rows where motif within an msp
msp_conds = (df["centered_position_type"] == "msp") & (df["centered_start"] <= 0) & (df["centered_end"] >= 35)
print("msps with a motif: {}".format(msp_conds.sum()))

# get group names (keys)
group_names = list(df_grouped.groups.keys())
print("different motif-sequence groups: {}".format(len(group_names)))

In [ ]:
len(df[msp_conds].groupby(grouping_cols).groups.keys())

__have to do regions within an MSP AND unique query name__ cannot just do by motifs to account for multiple strands

In [ ]:
print("msp's: {}".format(df[df["centered_position_type"] == "msp"].shape[0]))
print("m6a's: {}".format(df[df["centered_position_type"] == "m6a"].shape[0]))

In [ ]:
msp_conds = (df["centered_position_type"] == "msp") & (df["centered_start"] <= 0) & (df["centered_end"] >= 35)
print("msps with a motif: {}".format(msp_conds.sum()))

df[msp_conds].groupby(grouping_cols)

logic
* Motif first &rarr; for each motif, if a fiber does not have an MSP, remove that motif-fiber instance
* Fiber first &rarr;  for each query name, if a MSP does not contain a motif, remove the entire fiber

In [ ]:
# preserve original df
#df_v0 = df

In [ ]:
# test grouped df
df_t = df[(df["motif_name"] == "chr1_1779210_+") & (df["query_name"] == "m54329U_210323_190418/114558801/ccs")]
print("Total rows: {}".format(df_t.shape[0]))

In [ ]:
df_t[df_t["centered_position_type"] == "msp"]

In [ ]:
df_t.query("centered_position_type == 'msp'")["centered_start"] <= 0

In [ ]:
df_t.query("centered_position_type == 'msp'")["centered_end"] >= 35

In [ ]:
df_grouped.get_group(group_names[1])

In [ ]:
i = 0
for g_idx, group in df_grouped:
    print(g_idx)
    print(group.shape)
    if (group.query("centered_position_type == 'msp'")["centered_start"] <= 0 and 
        group.query("centered_position_type == 'msp'")["centered_end"] >= 35):
        print(group.query("centered_position_type == 'msp'"))
        for r_idx, row in group.iterrows():
            print(row)
    i += 1
    if i >= 2:
        break

In [ ]:
np.where()

In [ ]:
df.head(1)

In [ ]:
msp_conds = (df["centered_position_type"] == "msp") & (df["centered_start"] <= 0) & (df["centered_end"] >= 35)
print("msps with a motif: {}".format(msp_conds.sum()))

df[msp_conds].groupby()

In [ ]:
# for each group, add msp size


In [ ]:
len(list(df[msp_conds].groupby(grouping_cols).groups.keys()))

In [ ]:
msp_conds[0].sum()

In [ ]:
within_msp = []

msp_conds = [(df["centered_position_type"] == "msp")]

for group in group_names:
    d = df_grouped.get_group(group)
    d_msp = d[d["centered_position_type"] == "msp"]
    

In [ ]:
df_grouped.get_group(group_names[0]).tail()

In [ ]:
# create mask of MSPs containing a motif
msp_mask = df[df["centered_position_type"] == "msp"].apply(lambda row: (row["centered_start"] <= 0) & (row["centered_end"] >= 35), axis=1)

In [ ]:
msp_mask.value_counts()

In [ ]:
# indicies of msp's without a motif
msp_drop = df[df["centered_position_type"] == "msp"].index[~msp_mask]

In [ ]:
df.loc[msp_drop].motif_name.unique()

In [ ]:
df[df["motif_name"].isin(msp_motifs)]

In [ ]:
# remove msp's without a motif
df = df.drop(msp_drop)

In [ ]:
print(df.shape[0])

In [ ]:
# motif name can have multiple msps (1 msp per strand)
# query can have multiple msp's (different motifs)

# names of motifs within an msp
msp_motifs = df[df["centered_position_type"] == "msp"].motif_name.unique()
print("Unique motifs: {}".format(len(msp_motifs)))

In [ ]:
# indicies of msp's with a motif
df.loc[df[df["centered_position_type"] == "msp"]].motif_name.unique()

In [ ]:
# get motif_name's of motifs within an msp
print("Unique motifs with an msp: {}".format(df[df["centered_position_type"] == "msp"].motif_name[msp_mask].nunique()))

In [ ]:
# get motif_name's of motifs within an msp
df[df["centered_position_type"] == "msp"].motif_name[msp_mask].unique()

In [ ]:
# remove non-MSP groups
group_names = list(itertools.compress(group_names, msp_mask)

In [ ]:
# get msps that contain a motif
df_msp[df[df["centered_position_type"] == "msp"].apply(lambda row: (row["centered_start"] <= 0) & (row["centered_end"] >= 35), axis=1)]

In [ ]:
df[df["centered_position_type"] == "msp"].apply(lambda row: row["centered_end"] - row["centered_start"], axis=1))

In [ ]:
%%time
msp_len = np.array(df[df["centered_position_type"] == "msp"].apply(lambda row: row["centered_end"] - row["centered_start"], axis=1))

In [ ]:
df[np.array(df.apply(lambda row: (row.centered_position_type == "m6a") & 
                    (row.centered_start >= -40) & (row.centered_end <= 75), axis=1), dtype=bool)]

In [ ]:
# only msps
df_msp = df[df["centered_position_type"] == "msp"]
df_msp.shape

In [ ]:
len(list(df_msp[(df_msp["centered_start"] <= 0) & (df_msp["centered_end"] >= 35)].groupby(grouping_cols).groups.keys()))

In [ ]:
print("mean msp size:", np.mean(msp_len))
print("median msp size", np.median(msp_len))
print("min msp size:", np.min(msp_len))
print("max msp size:", np.max(msp_len))

In [ ]:
df.iloc[:0]

__filter m6as for 40 bp regions flanking motif__

In [ ]:
%%time
df = df[(df["centered_start"] >= -40) & (df["centered_end"] <= 75)]
print("rows within bounds: {}".format(df.shape[0]))

In [ ]:
def within_range(df, position_type=["msp", "m6a"], lower_bound=-40, upper_bound=75):
    ''' Check if row coord are within a range. '''
    msp_mask = (df.centered_position_type.isin(position_type)) & (df.centered_start >= lower_bound) & (df.centered_end <= upper_bound)
    return np.array(msp_mask, dtype=bool)

In [ ]:
grouping_cols = df.columns.tolist()[:6]

# group by first 6 columns
df_grouped = df.groupby(grouping_cols)

# get group names (keys)
group_names = list(df_grouped.groups.keys())
print("different motif-sequence groups: {}".format(len(group_names)))

__filtering regions for MSPs__

In [ ]:
# check if rows of a certain type are outside a range
def outside_range(df, position_type=["msp"], lower_bound=0, upper_bound=35):
    ''' Check if row coord are beyond a range. '''
    msp_mask = (df.centered_position_type.isin(position_type)) & (df.centered_start <= lower_bound) & (df.centered_end >= upper_bound)
    return np.array(msp_mask, dtype=bool)

* if there is no MSP from 0-35, drop that motif & strand group
* if there IS an MSP (only 1 per motif)
 - drop m6a's not within MSP bounds
 - only get m6a sites within the MSP

In [ ]:
%%time
group_names = list(df_grouped.groups.keys())
print("Total groups: {}".format(len(group_names)))

# mask to remove non-MSP groups
msp_mask = []

for group in group_names:
    d = df_grouped.get_group(group)
    msp_mask.append(outside_range(d).any())

print("Groups within an MSP: {}".format(sum(msp_mask)))
print("Groups outside an MSP: {}".format(len(group_names) - sum(msp_mask)))
# remove non-MSP groups
group_names = list(itertools.compress(group_names, msp_mask))

In [ ]:
len(group_names)

In [ ]:
df_grouped.get_group(group_names[10])

In [ ]:
%%timeit

group_names = list(df_grouped.groups.keys())
#print("Total groups: {}".format(len(group_names)))

# mask to remove non-MSP groups
msp_mask = []

for group in group_names:
    d = df_grouped.get_group(group)
    msp_mask.append(outside_range(d).any())

#print("Groups within an MSP: {}".format(sum(msp_mask)))
#print("Groups outside an MSP: {}".format(len(group_names) - sum(msp_mask)))
# remove non-MSP groups
group_names = list(itertools.compress(group_names, msp_mask))

In [ ]:
158/5400 * 20053779 * 0.001

In [ ]:
586.7587188888889 / 60

In [ ]:
d = dfs_within_msp[7]
print(d.shape)
d.loc[d["centered_position_type"] == "msp", ["centered_position_type", "centered_start", "centered_end"]].shape[0]

In [ ]:
for ix, g in enumerate(groups_within_msp):
    d = get_grouped_df(df_grouped, g)
    if d.loc[d["centered_position_type"] == "msp", ["centered_start", "centered_end"]].shape[0] > 1:
        print(ix)

In [ ]:
get_grouped_df(df_grouped, groups_within_msp[100])[["centered_position_type", "centered_start", "centered_end"]]

In [ ]:
d.loc[get_in_range(d), ["centered_start", "centered_end"]].to_numpy()

In [ ]:
np.array(d.apply(lambda row: (row.centered_position_type == "msp") & 
                    (row.centered_start <= 0) & (row.centered_end >= 45), axis=1), dtype=bool).any()

In [ ]:
d[np.array(d.apply(lambda row: (row.centered_position_type == "msp") & 
                    (row.centered_start <= 0) & (row.centered_end >= 35), axis=1), dtype=bool)]

In [ ]:
# get df of specific group
def get_grouped_df(grouped_df, group_name):
    ''' Get df group from grouped df. '''
    return df_grouped.get_group(group_name)

In [ ]:
%%time
print(len(groups_outside_msp))
dfs_outside_msp= [get_grouped_df(grouped_df=df_grouped, group_name=name) for name in groups_outside_msp]
print(len(dfs_outside_msp))

In [ ]:
dfs_outside_msp = [df[df["centered_position_type"] == "msp"] for df in dfs_outside_msp]

In [ ]:
dfs_outside_msp[0]

__filter for 40 bps flanking left & right of motif__

In [ ]:
sys.getsizeof(df_grouped)

In [ ]:
%%time
dfs_within_msp = [get_grouped_df(grouped_df=df_grouped, group_name=name) for name in groups_within_msp]

In [ ]:
d = dfs_within_msp[0]
print(d.shape)
d.loc[d["centered_position_type"] == "msp", ["centered_position_type", "centered_start", "centered_end"]]

In [ ]:
d

In [ ]:
d[d["centered_position_type"] == "msp"][d[d["centered_position_type"] == "msp"].apply(lambda row: (row.centered_position_type == "msp") & 
                    (row.centered_start <= 0) & (row.centered_end >= 35), axis=1)]

In [ ]:
row_0 = df[df.columns[:6]].iloc[0]

In [ ]:
# group by first 6 columns (first 3 cols: for each motif, last 3 cols: for each fiber)
df_grouped = df.groupby(df.columns.tolist()[:6], sort=False)

In [ ]:
df_grouped.head(5)

In [ ]:
group_keys = df_grouped.groups.keys()

In [ ]:
for name, group in df_grouped:
    print(name)
    print(group)

In [ ]:
df.groupby(df.columns.tolist()[:6], sort=False).apply(lambda row: row[(row["centered_start"] <= 0) & (row["centered_end"] >= 35)].sum())

In [ ]:
# filter motifs for those inside MSPs

In [ ]:
df.groupby(df.columns.tolist()[:6], sort=False, group_keys=True).first()

In [ ]:
df["query_name"][7000]

In [ ]:
df.nunique()

In [ ]:
# per motif per fiber/read
idx_t = 7000
df_0 = df[(df["chrom"] == df["chrom"][idx_t]) &  
(df["centering_position"] == df["centering_position"][idx_t]) & 
(df["strand"] == df["strand"][idx_t]) & 
    #(df["subset_sequence"] == df["subset_sequence"][idx_t]) &
    # #(df["reference_start"] == df["reference_start"][idx_t]) & 
    # #(df["reference_end"] == df["reference_end"][idx_t]) & 
(df["query_name"] == df["query_name"][idx_t])]
df_0.shape

In [ ]:
df_0

In [ ]:
# create feature table
cols = ["chrom", "centering_position", "strand", "query_name", "subset_sequence", "left_m6a_count", "right_m6a_count"]

In [ ]:
# step 1. filter for motif within MSP
df_0[df_0["centered_position_type"] == "msp"]

In [ ]:
# step 1. filter for motif within MSP
df_0[(df_0["centered_position_type"] == "msp") & (df_0["centered_start"] <= 0) & (df_0["centered_end"] >= 35)]

In [ ]:
df_0[df_0["centered_position_type"] == "msp"][["centered_start", "centered_end"]]

In [ ]:
df_0

In [ ]:
# filter for msps
df_0[["centered_start", "centered_end"]].apply(lambda row: (row.centered_start <= 0) & (row.centered_end >= 35), axis=1).any()

In [ ]:
def filt_for_msps(row):
    return lambda row: (row.centered_start <= 0) & (row.centered_end >= 35)

In [ ]:
df_0.apply(filt_for_msps, axis=1)

In [ ]:
def filt_for_msps(row):
    '''Filter for regions within MSPs.'''
    print(row)
    print(row.centered_start[0])
    if ((row.centered_start <= 0) & (row.centered_end >= 35)).any():
        True
    else:
        False

In [ ]:
df_0[df_0["centered_position_type"] == "msp"]

In [ ]:
np.where((df_0[df_0["centered_position_type"] == "msp"]["centered_start"] <= 0) & (df_0[df_0["centered_position_type"] == "msp"]["centered_end"] >= 100), True, False)

In [ ]:
(df_0[df_0["centered_position_type"] == "msp"]["centered_start"] <= 0) & (df_0[df_0["centered_position_type"] == "msp"]["centered_end"] >= 100)

In [ ]:
df_t = df_0[df_0["centered_position_type"] == "m6a"]

In [ ]:
df_grouped = df.groupby(df.columns.tolist()[:3])

In [ ]:
df_grouped.head()

In [ ]:
df_grouped[["centered_position_type", "centered_start","centered_end"]].apply(lambda row: row.centered_position_type == "msp")

In [ ]:
df_0.apply(filt_for_msps(df_0[df_0["centered_position_type"] == "msp"]), axis=1)